Date: 27/10/2019

Version: 2.0

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

Libraries used:
* pandas 0.19.2 (for data frame, included in Anaconda Python 3.6) 
* re 2.2.1 (for regular expression, included in Anaconda Python 3.6) 
* requests (for url extraction, included in Anaconda Python 3.6)
* os (for operation, included in Anaconda Python 3.6)
* matplotlib (for plotting, included in Anaconda Python 3.6)

In [1]:
# import libraries
import pandas as pd
import geopandas as gpd
from geopandas import *
import requests
import zipfile
from zipfile import ZipFile
from io import *
import matplotlib.pyplot as plt
import xlrd
%matplotlib inline
import re
import os

### Unzip Files to Created Folders

The **'vic_suburb_boundary.zip'** file is extracted to a newly built folder named **'boundPath'**. 

In [ ]:
# build new folder to store files from vic_suburb_boundary.zip
bound_path = './boundPath'
bound = os.makedirs(bound_path)


In [ ]:
# unzip the suburb file √
zipped = ZipFile('vic_suburb_boundary.zip')
zipped.extractall(path=bound_path)


The **gtfs.zip** file is extracted to another newly built folder **gtfs** 

In [ ]:
# make new folder to store extracted gtfs
gtfs = os.makedirs('./gtfs')

In [ ]:
# path of the big zip file
gt_root = './gtfs'

# open the big zip file
a = zipfile.ZipFile('gtfs.zip') 
a.extractall(path = gt_root)


In [ ]:
# get list of folders in the big zip
n_list = a.namelist()

# collect the directory of zips
eff_list =[] 
for nm in n_list:
    if re.search(r'\.zip$', nm) != None:
        eff_list.append(gt_root + '/' + nm)


There are 11 sub-folders in extracted gtfs folder. Each sub-folder includes a small zip file. Extract the files in sub-folder 1 and 2 to folder **transit1** and **transit2** respectively. 

In [ ]:
# make new dir to store transit info in gtfs/1
local_path1 = './transit1'
os.makedirs(local_path1)

# make new dir to store transit info in gtfs/2
local_path2 = './transit2'
os.makedirs(local_path2)

In [ ]:
# extract file in folder 1
n1 = eff_list[2]

# extract the files to transit1 folder
ctt1 = zipfile.ZipFile(n1)
ctt1.extractall(path=local_path1)


In [ ]:
# extract file in folder 2
n2 = eff_list[3]

# extract the files to transit2 folder
ctt2 = zipfile.ZipFile(n2)
ctt2.extractall(path=local_path2)


### Read Files and Fix Column Names

In [2]:
# read group csv
gr = pd.read_csv('Group108.csv')

In [3]:
# fix errors in column names
gr.rename(columns = {"Landsize":"LandSize", "Lattitude":"Latitude", "Longtitude":"Longitude"}, inplace=True)

In [4]:
# add columns with default values
gr['Suburb'] = "not available"
gr['train_station_id'] = -1
gr['distance_to_train_station'] = -1
gr['travel_min_to_CBD'] = -1
gr['over_priced'] = -1
gr['crime_A_average']= -1
gr['crime_B_average']= -1
gr['crime_C_average']= -1
gr['closest_primary_school']= "not available"
gr['distance_to_closest_primary']= -1
gr['primary_school_ranking']= -1
gr['closest_secondary_school']= "not available"
gr['distance_to_closest_secondary']= -1
gr['secondary_school_ranking']= -1

In [5]:
# view dataframe
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,over_priced,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,-1,-1,-1,-1,not available,-1,-1,not available,-1,-1
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,-1,-1,-1,-1,not available,-1,-1,not available,-1,-1
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,-1,-1,-1,-1,not available,-1,-1,not available,-1,-1
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,-1,-1,-1,-1,not available,-1,-1,not available,-1,-1
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,-1,-1,-1,-1,not available,-1,-1,not available,-1,-1


#### Read shape files for suburbs

In [6]:
# list all files under the foler of suburb boundary
sub_names = os.listdir('./boundPath')
sub_names

['VIC_LOCALITY_POLYGON_shp.dbf',
 'VIC_LOCALITY_POLYGON_shp.prj',
 'VIC_LOCALITY_POLYGON_shp.shp',
 'VIC_LOCALITY_POLYGON_shp.shx']

In [7]:
# read suburb file
shp_rd = gpd.read_file('boundPath' + '/' + sub_names[2])
shp_rd.head(3)

,LC_PLY_PID,DT_CREATE,DT_RETIRE,LOC_PID,VIC_LOCALI,VIC_LOCA_1,VIC_LOCA_2,VIC_LOCA_3,VIC_LOCA_4,VIC_LOCA_5,VIC_LOCA_6,VIC_LOCA_7,geometry
0,6670,2011-08-31,None,VIC2615,2012-04-27,None,UNDERBOOL,None,None,G,None,2,"POLYGON ((141.74552399 -35.07228701, 141.74552..."
1,6671,2011-08-31,None,VIC1986,2012-04-27,None,NURRAN,None,None,G,None,2,"POLYGON ((148.668767 -37.39571245, 148.6687620..."
2,6672,2011-08-31,None,VIC2862,2012-04-27,None,WOORNDOO,None,None,G,None,2,"POLYGON ((142.92287999 -37.97885997, 142.90449..."


In [8]:
from shapely.geometry import Point

In [9]:
lon = gr['Longitude']
lat = gr['Latitude']

gr['Points'] = GeoSeries([Point(x,y) for x,y in zip(lon, lat)])
gr.head(3)

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9339 -37.7835)


In [10]:
from shapely.geometry import Point, Polygon

In [11]:
# list of suburb based on ID
sub_list = []

ls_pts = gr['Points']

ls_subs = shp_rd['VIC_LOCA_2']

ls_geo = shp_rd['geometry'].tolist()

In [12]:
# define a function to match point
def check_geometry(pt, pl_df):
    
    for i in range(len(pl_df)):
        rg = pl_df[i] 
        # check the point in polygon
        if pt.within(rg) == True:
            return i
        else:
            continue
    return None

In [13]:

# collect the suburbs
for i in gr.index.values.tolist():
    # position of each point 
    pt = ls_pts[i]
    # find index of suburbs
    ids = check_geometry(pt, ls_geo)
    gr.loc[i,'Suburb'] = ls_subs[ids]
        

#### train Station ID

#### Read in information related 

In [14]:
# create the first stop dataframe
stop1 = pd.read_csv('./transit1/stops.txt')
stop1.head(3)

,stop_id,stop_name,stop_lat,stop_lon
0,17204,Wallan Railway Station (Wallan),-37.416861,145.005372
1,19980,Melton Railway Station (Melton South),-37.703293,144.572524
2,19981,Rockbank Railway Station (Rockbank),-37.729261,144.650631


In [15]:
# create the second stop dataframe
stop2 = pd.read_csv('./transit2/stops.txt')
stop2.head(3)

,stop_id,stop_name,stop_lat,stop_lon
0,15351,Sunbury Railway Station (Sunbury),-37.579091,144.727319
1,15353,Diggers Rest Railway Station (Diggers Rest),-37.627017,144.719922
2,19827,Stony Point Railway Station (Crib Point),-38.374235,145.221837


In [16]:
# create the first calendar dataframe
calendar1 = pd.read_csv('./transit1/calendar.txt')
calendar1.head(3) 

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,UR,0,0,0,1,0,0,0,20190509,20190509
1,T6,0,0,0,0,1,0,0,20190510,20190510
2,T2,0,0,0,0,0,1,0,20190511,20190511


In [17]:
# create the second calendar dataframe
calendar2 = pd.read_csv('./transit2/calendar.txt')
calendar2.head(3) 

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,T5,0,0,0,1,0,0,0,20190509,20190509
1,UJ,0,0,0,0,0,0,1,20190510,20190512
2,T2,0,0,0,0,0,1,0,20190510,20190512


In [18]:
# create the first stop-time dataframe
stop_times1 = pd.read_csv('./transit1/stop_times.txt')
stop_times1.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,7548.UR.1-V08-A-mjp-1.1.H,06:08:00,06:08:00,20291,1,NaN,0,0,0.000000
1,7548.UR.1-V08-A-mjp-1.1.H,06:42:00,06:42:00,20346,2,NaN,0,0,52720.840026
2,7548.UR.1-V08-A-mjp-1.1.H,06:55:00,06:57:00,20341,3,NaN,0,0,68786.649688


In [19]:
# create the second stop-time dataframe
stop_times2 = pd.read_csv('./transit2/stop_times.txt')
stop_times2.head(3)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,9117.T5.2-ALM-A-mjp-1.1.H,05:21:00,05:21:00,19853,1,NaN,0,0,0.000000
1,9117.T5.2-ALM-A-mjp-1.1.H,05:24:00,05:24:00,19852,2,NaN,0,0,1506.610171
2,9117.T5.2-ALM-A-mjp-1.1.H,05:25:00,05:25:00,19851,3,NaN,0,0,1992.788001


In [20]:
# create the first trip dataframe
trips1 = pd.read_csv('./transit1/trips.txt')
trips1.head(3)

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,1-V08-G-mjp-1,T0,1051.T0.1-V08-G-mjp-1.1.H,1-V08-G-mjp-1.1.H,Melbourne,0
1,1-V08-G-mjp-1,T0,1052.T0.1-V08-G-mjp-1.2.H,1-V08-G-mjp-1.2.H,Melbourne,0
2,1-V08-G-mjp-1,T0,1053.T0.1-V08-G-mjp-1.2.H,1-V08-G-mjp-1.2.H,Melbourne,0


In [21]:
# create the second trip dataframe
trips2 = pd.read_csv('./transit2/trips.txt')
trips2.head(3)

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,2-ALM-F-mjp-1,T0,8262.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,Alamein,0
1,2-ALM-F-mjp-1,T0,8267.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,Alamein,0
2,2-ALM-F-mjp-1,T0,8271.T0.2-ALM-F-mjp-1.1.H,2-ALM-F-mjp-1.1.H,Alamein,0


#### For files from folder 1, select working train that works on weekdays

In [22]:
# find service ID that works through weekdays
working_ids = calendar1.loc[(calendar1['monday']==1) \
                            & (calendar1['tuesday']==1) \
                            & (calendar1['wednesday']==1) \
                            & (calendar1['thursday']==1) \
                            & (calendar1['friday']==1)]
working_ids.head(3)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
6,T0,1,1,1,1,1,0,0,20190515,20190523
13,T0_1,1,1,1,1,1,0,0,20190530,20190606
16,T0_2,1,1,1,1,1,0,0,20190607,20190628


In [23]:
# take the service id of weekdays to a list
check_ids = list(working_ids['service_id'])
check_ids

['T0', 'T0_1', 'T0_2', 'T0_3']

In [24]:
# filter the dataframe based on service id
have_trip = trips1.loc[trips1.service_id.isin(check_ids) == True]
have_trip.head(3)

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
0,1-V08-G-mjp-1,T0,1051.T0.1-V08-G-mjp-1.1.H,1-V08-G-mjp-1.1.H,Melbourne,0
1,1-V08-G-mjp-1,T0,1052.T0.1-V08-G-mjp-1.2.H,1-V08-G-mjp-1.2.H,Melbourne,0
2,1-V08-G-mjp-1,T0,1053.T0.1-V08-G-mjp-1.2.H,1-V08-G-mjp-1.2.H,Melbourne,0


#### Find routines passes Southern Cross Station

Filter the service id in table of trip for service work on weekdays, which is working_ids. 
In stop-time table, the stop ids of Southern Cross station are found and find the corresponding trip-ids.
Then use the trip ids to locate stop ids. The stop ids are later used to match the location of each station.

In [25]:
# find out the stop ID of Southern Cross Station 
south_cross = stop1.loc[stop1.stop_name == 'Southern Cross Railway Station (Melbourne City)']
stopId_Southern = list(south_cross.stop_id)[0]
stopId_Southern

20043

In [26]:
# find trip routines through Southern Cross
# exclude the Southern Cross station itself
southern_selection = stop_times1.loc[(stop_times1.stop_id == stopId_Southern) \
                                     & (stop_times1.stop_sequence != 1)] 
southern_selection.head()
trip_id_list = list(southern_selection['trip_id'])
# trip_id_list

In [27]:
# find stop id based on selected trip ids
part_onRoute = stop_times1.loc[(stop_times1.trip_id.isin(trip_id_list) == True) \
                             & (stop_times1.departure_time >= '07:00:00')\
                             & (stop_times1.departure_time <= '09:30:00')]
# part_onRoute.head()
stopIds_onRoute = list(set(part_onRoute.stop_id))
# store the stopids in a list
tripIds_onRoute = list(set(part_onRoute.trip_id))


In [28]:
# obtain trip id and route id in filtered trips table
trip_info1 = have_trip.loc[have_trip.trip_id.isin(tripIds_onRoute) == True]

In [29]:
# filter stops and corresponding location 
stop_location1 = stop1.loc[stop1.stop_id.isin(stopIds_onRoute)]
stop_location1.describe()

,stop_id,stop_lat,stop_lon
count,98.000000,98.000000,98.000000
mean,22218.469388,-37.535539,144.902509
std,6608.294925,0.656183,0.940363
min,17204.000000,-38.386394,142.475545
25%,20304.250000,-38.089522,144.360571
50%,20329.500000,-37.742636,144.817533
75%,20356.750000,-37.194105,145.355090
max,52011.000000,-35.733122,147.627614


#### Do the same steps for folder 2 again

In [30]:
# find service ID that works through weekdays
working_ids2 = calendar2.loc[(calendar1['monday']==1) \
                            & (calendar1['tuesday']==1) \
                            & (calendar1['wednesday']==1) \
                            & (calendar1['thursday']==1) \
                            & (calendar1['friday']==1)]
working_ids2.head(3)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
6,T5+td,1,1,1,0,0,0,0,20190513,20190516
13,UJ_2,0,0,0,0,0,0,1,20190524,20190810
16,T5+ll,0,1,1,0,0,0,0,20190513,20190516


In [31]:
# take the service id of weekdays to a list
check_ids2 = list(working_ids2['service_id'])
check_ids2

['T5+td', 'UJ_2', 'T5+ll', 'T0+a5']

In [32]:
# filter the dataframe based on service id
have_trip2 = trips2.loc[trips2.service_id.isin(check_ids2) == True]
have_trip2.head(3)

,route_id,service_id,trip_id,shape_id,trip_headsign,direction_id
2360,2-BEL-F-mjp-1,T0+a5,9281.T0.2-BEL-F-mjp-1.11.H,2-BEL-F-mjp-1.11.H,Belgrave,0
2361,2-BEL-F-mjp-1,T0+a5,8891.T0.2-BEL-F-mjp-1.22.R,2-BEL-F-mjp-1.22.R,City (Flinders Street),1
2362,2-GLW-F-mjp-1,T0+a5,8650.T0.2-GLW-F-mjp-1.5.H,2-GLW-F-mjp-1.5.H,Glen Waverley,0


In [33]:
# find out the stop ID of Southern Cross Station 
south_cross2 = stop2.loc[stop2.stop_name == 'Southern Cross Railway Station (Melbourne City)']
stopId_Southern2 = list(south_cross2.stop_id)[0]
stopId_Southern2

22180

In [34]:
# find trip routines through Southern Cross
southern_selection2 = stop_times2.loc[(stop_times2.stop_id == stopId_Southern2) \
                                     & (stop_times2.stop_sequence != 1)]
# southern_selection.head()
trip_id_list2 = list(southern_selection2['trip_id'])
# print(len(trip_id_list2))
# trip_id_list2[0:10]

In [35]:
# find stop id based on selected trip ids
part_onRoute2 = stop_times2.loc[(stop_times2.trip_id.isin(trip_id_list2) == True) \
                               & (stop_times2.departure_time >= '07:00:00') \
                                & (stop_times2.departure_time <= '09:30:00')]
# store the stopids in a list
stopIds_onRoute2 = list(set(part_onRoute2.stop_id))
# print(len(stopIds_onRoute2))
tripIds_onRoute2 = list(set(part_onRoute2.trip_id))

In [36]:
# obtain trip id and route id in filtered trips table
trip_info2 = have_trip2.loc[have_trip2.trip_id.isin(tripIds_onRoute2) == True]

In [37]:
# filter stop location by 
stop_location1 = stop1.loc[stop1.stop_id.isin(stopIds_onRoute) == True]
# stop_location1.head(3)

In [38]:
stop_location2 = stop2.loc[stop2.stop_id.isin(stopIds_onRoute2)]
# stop_location2.head(3)

In [39]:
# merge two train stop tables together
table_stops = pd.concat([stop_location1,stop_location2], ignore_index=True)
table_stops.head(3)

,stop_id,stop_name,stop_lat,stop_lon
0,17204,Wallan Railway Station (Wallan),-37.416861,145.005372
1,19980,Melton Railway Station (Melton South),-37.703293,144.572524
2,19981,Rockbank Railway Station (Rockbank),-37.729261,144.650631


In [ ]:
# len(set(list(table_stops.stop_id)))

#### Find closest train station and distance to station

In [40]:
from math import radians, cos, sin, asin, sqrt

In [41]:
# find distance between two points
def haversine(lat1, lon1, lat2, lon2):

    radius = 6378

    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    lat_diff = lat2 - lat1
    lon_diff = lon2 - lon1

    a = sin(lat_diff/2)**2 + cos(lat1)*cos(lat2)*sin(lon_diff/2)**2
    c = 2*asin(sqrt(a))
    
    distance = radius*c

    return distance


In [42]:
# longitude and latitude of properties
sub_lat = gr.Latitude.tolist()
sub_lon = gr.Longitude.tolist()

# longitude and latitude of stations
station_lat = table_stops.stop_lat.tolist()
station_lon = table_stops.stop_lon.tolist()

In [43]:
# Function to find minimum distance
# lon2, lat2 为房子的经纬度
def find_distance_station(lon2, lat2, station_df):
    
    # 车站经度列表
    list_lon1 = station_df['stop_lon']
    # 车站纬度列表
    list_lat1 = station_df['stop_lat']
    list_station = station_df['stop_id']
    
    # build empty list for further use
    # 距离空表
    distance_list = []
    
    for i in station_df.index.values.tolist():
        lon1 = list_lon1[i]
        lat1 = list_lat1[i]
        # 收集所有车站到该住宅的距离
        distance_list.append(haversine(lat1, lon1, lat2, lon2))  
        
    
    # 获得最短距离 -- 
    min_distance = min(distance_list)
    
    # 获取最短距离的index
    distance_index = distance_list.index(min_distance)
    
    # 用tuple 收集index和最短距离
    min_distance_index = (distance_index, min_distance)
    
    return min_distance_index

In [44]:
# use function to find minimum distance
for i in gr.index.values.tolist():
    
    # 遍历每一个property
    lon2 = sub_lon[i]
    lat2 = sub_lat[i]
    
    # get index of station
    station_index = find_distance_station(lon2, lat2, table_stops)[0]
    # get stop id by index
    gr.loc[i,'train_station_id'] = table_stops.loc[station_index, 'stop_id']
    
    # get distance between station and property
    gr.loc[i,'distance_to_train_station'] = find_distance_station(lon2, lat2, table_stops)[1]
   

In [45]:
# check dataframe
gr.head(3)

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9339 -37.7835)


#### Get travel minutes to CBD

For both the trip tables, the index of table is reset by row. New columns of departure hour and departure minutes are generated. The hour column is multiplied by 60 and added to minutes.


#### find travel minutes for trip in folder 1

In [ ]:
# reset the dataframe in selected data 
part_onRoute = part_onRoute.reset_index(drop=True)


In [ ]:
# create hour and minute columns in the dataframe
part_onRoute['departure_hour'] = -1
part_onRoute['departure_min'] = -1
part_onRoute.head(3)

In [ ]:
# insert two new columns of hours and minutes

for i in part_onRoute.index.values.tolist():
    arr_time = part_onRoute['departure_time'].tolist()[i]
    # extract hours
    hours = int(arr_time[0:2])
    # extract minutes
    mins = int(arr_time[3:5])
    # add new hours to dataframe
    part_onRoute.loc[i,'departure_hour'] = hours
    # add new minutes to dataframe
    part_onRoute.loc[i,'departure_min'] = mins


In [ ]:
part_onRoute.head(3)

In [ ]:
# 每一行小时+分钟转换时间
part_onRoute['time'] = part_onRoute.apply(lambda x: x['departure_hour']*60 +x['departure_min'], axis=1)

In [ ]:
# StopId of Southern Cross Station
stopId_Southern

The stop id of Southern Cross station has been found previously. The sequence of this station can be identified with stop id for each route. 


In [ ]:
#the sequence of southern cross on each route 
cross_data = part_onRoute.loc[part_onRoute['stop_id']==stopId_Southern] #.reset_index(drop=True)

# sequence to list  
cross_seq = cross_data['trip_id'].tolist()

The sequence of each stop for each route can be compared to the sequence of Southern Cross Station, to ensure the stop is before Southern Cross on the route. 

The travel minutes are equal to the departure time between each filtered stop and Southern Cross Station


In [ ]:
# assign value of travel minute to column 
for i in part_onRoute.index.values.tolist():
    #trip_id
    route = part_onRoute.loc[i, 'trip_id']
    #stop_sequence
    route_seq = part_onRoute.loc[i, 'stop_sequence']
    
    # If southern cross in the route
    if route not in cross_seq:
        part_onRoute[i,'travel_min'] = -1

    elif route in cross_seq:
        # trip_id index
        south_index = part_onRoute.loc[(part_onRoute['trip_id']==route) \
                                 & (part_onRoute['stop_id']==stopId_Southern)].index.tolist()[0]
        
        # sequence of Souther Cross
        south_seq = part_onRoute.loc[south_index,'stop_sequence']
    
        if route_seq < south_seq:
            part_onRoute[i,'travel_min'] = part_onRoute.loc[south_index,'time'] - part_onRoute.loc[i,'time'] 
        else:
            part_onRoute[i,'travel_min'] = -1

#### find travel minutes for trip in folder 2
The process of dealing with travel minutes for trip2 table is the same with the process of travel minutes of trip1 above. 

In [ ]:
# reset the dataframe in selected data 
part_onRoute2 = part_onRoute2.reset_index(drop=True)


In [ ]:
# create hour and minute columns in the dataframe
part_onRoute2['departure_hour'] = -1
part_onRoute2['departure_min'] = -1
part_onRoute2.head(3)

In [ ]:
# insert two new columns of hours and minutes

for i in part_onRoute2.index.values.tolist():
    arr_time = part_onRoute2['departure_time'].tolist()[i]
    # extract hours
    hours = int(arr_time[0:2])
    # extract minutes
    mins = int(arr_time[3:5])
    # add new hours to dataframe
    part_onRoute2.loc[i,'departure_hour'] = hours
    # add new minutes to dataframe
    part_onRoute2.loc[i,'departure_min'] = mins


In [ ]:
# 每一行小时+分钟转换时间
part_onRoute2['time'] = part_onRoute2.apply(lambda x: x['departure_hour']*60 +x['departure_min'], axis=1)

In [ ]:
#the sequence of southern cross on each route 
cross_data2 = part_onRoute2.loc[part_onRoute2['stop_id']==stopId_Southern2] #.reset_index(drop=True)

# sequence to list  
cross_seq2 = cross_data2['trip_id'].tolist()

In [ ]:
part_onRoute2.head(3)

In [ ]:
# assign value of travel minute to column 
for i in part_onRoute2.index.values.tolist():
    #trip_id
    route = part_onRoute2.loc[i, 'trip_id']
    #stop_sequence
    route_seq = part_onRoute2.loc[i, 'stop_sequence']
    
    print(i)
    
    # if the route has southern cross
    if route not in cross_seq2:
        part_onRoute2[i,'travel_min'] = -1

    elif route in cross_seq2:
        # index of tripid
        south_index = part_onRoute2.loc[(part_onRoute2['trip_id']==route) \
                                 & (part_onRoute2['stop_id']==stopId_Southern2)].index.tolist()[0]
        
        # sequence on the route
        south_seq = part_onRoute2.loc[south_index,'stop_sequence']
    
        if route_seq < south_seq:
            part_onRoute2[i,'travel_min'] = part_onRoute2.loc[south_index,'time'] - part_onRoute2.loc[i,'time'] 
        else:
            part_onRoute2[i,'travel_min'] = -1

Concatenate the modified tables together with column index revised.

In [ ]:
total_trip = pd.concat([part_onRoute,part_onRoute2], ignore_index=True)
total_trip.head(3)

Match the travel minute and station id with each property

In [ ]:
# match travel min to each property

for i in gr.index.values.tolist():
    # Find the id of closest station
    closest_station = gr.loc[i, 'train_station_id']
    # filter all the travel minutes for the stop id
    travel_time = total_trip.loc[total_trip['stop_id'] == closest_station] 
    
    # calculate the average travel minute 
    time_list = travel_time['travel_min'].tolist()
    average_min = np.mean(time_list)
    gr[i, 'travel_min_to_CBD'] = average_min

# Find Boolean feature for Over Price

This step uses `groupby` and `numpy` to find out the median for each row, then compare the value, 1 represents true, which means sold price is higher than median price for this type of property.

In [46]:
#get year from df and add as new column
date = gr['Date']
select_year=list(date.str.extract(r'(\d{4})', expand = False))
gr['Year']=select_year
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points,Year
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0657 -37.79541),2017
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.76863 -37.74521),2017
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9339 -37.7835),2016
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9985 -37.8212),2016
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0889 -37.8555),2017


In [47]:
#use groupby to select the columns and calculate the median for same type
average_price=gr.groupby(['Rooms','Type','Bathroom','Car','Year','Suburb'])['Price'].apply(np.median)
average_price.name = 'MEDIAN'
gr=gr.join(average_price, on=['Rooms','Type','Bathroom','Car','Year','Suburb'])
#adding median as new column 

In [48]:
over_price=[]
price=gr['Price']
med=gr['MEDIAN']
# compare price column and median column, append in list
for i in range(len(price)):
    if price[i]>med[i]:
        over_price.append(int(price[i]>med[i]))
    else:
        over_price.append(int(False))
over_price

[0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [49]:
gr['over_priced'] = over_price
# delete columns added before
gr=gr.drop(columns=['Year', 'MEDIAN'])
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,-1,-1,-1,not available,-1,-1,not available,-1,-1,POINT (145.0889 -37.8555)


# Find Average Crime_A_B_C

The type A,B,C crime relates to council which suburb in and year, therefore I build three dictionary A,B,C to store the offence count for each council in each year. It is easier to find data for future use, and it does not need to change any coding.

In [50]:
#get suburb and year from df use re
date = gr['Date']
year=list(date.str.extract(r'(\d{4})', expand = False))
print(len(year))
suburb = gr['Suburb']
sub=list(suburb.str.extract(r'(.*)', expand = False))
print(len(sub))

720
720


In [51]:
#read xlms table1 then drop nan, leave the things we need
cr_path = 'crimebylocationdatatable-yearending31march2016.xlsx'
crime = pd.read_excel(cr_path, sheet_name=1)
#drop frist row
crime=crime.dropna(thresh=2)
#drop first column
crime=crime.dropna(axis='columns')
crime=crime.rename(columns=crime.iloc[0])
crime=crime.drop(crime.index[0])
crime=crime.reset_index(drop=True)
crime

,Apr - Mar reference period,Police Region,Police Service Area,Local Government Area,CSA Offence Division,CSA Offence Subdivision,Offence Count,LGA ERP,Victorian ERP
0,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A10 Homicide and related offences,4,123600,5632521
1,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A20 Assault and related offences,568,123600,5632521
2,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A30 Sexual offences,132,123600,5632521
3,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A40 Abduction and related offences,12,123600,5632521
4,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A50 Robbery,46,123600,5632521
5,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A60 Blackmail and extortion,2,123600,5632521
6,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,"A70 Stalking, harassment and threatening behav...",148,123600,5632521
7,2012,1 North West Metro,BANYULE,BANYULE,A Crimes against the person,A80 Dangerous and negligent acts endangering p...,75,123600,5632521
8,2012,1 North West Metro,BANYULE,BANYULE,B Property and deception offences,B10 Arson,53,123600,5632521
9,2012,1 North West Metro,BANYULE,BANYULE,B Property and deception offences,B20 Property damage,938,123600,5632521


In [52]:
#create three dictionary for sum of type A,B,C offence as value in each council area with year
#for example: {'2012_BANYULE': 987,'2012_BRIMBANK': 2112,'2012_DAREBIN': 1307}
all_sub=crime.iloc[:,3].unique()
list_year=crime.iloc[:,0].unique()
Year_sub_A={}
for each_year in list_year:
    sel_year_df=crime.loc[(crime['Apr - Mar reference period'] == each_year)]
    for each_sub in all_sub:
        #select each area with year
        final_df=sel_year_df.loc[(sel_year_df['Local Government Area'] == each_sub) & (sel_year_df['CSA Offence Division'] == 'A Crimes against the person')]
        total_A=final_df['Offence Count'].sum()
        name=str(each_year)+'_'+each_sub
        Year_sub_A[name]=total_A
Year_sub_B={}
for each_year in list_year:
    sel_year_df=crime.loc[(crime['Apr - Mar reference period'] == each_year)]
    for each_sub in all_sub:
        final_df=sel_year_df.loc[(sel_year_df['Local Government Area'] == each_sub) & (sel_year_df['CSA Offence Division'] == 'B Property and deception offences')]
        total_B=final_df['Offence Count'].sum()
        name=str(each_year)+'_'+each_sub
        Year_sub_B[name]=total_B
Year_sub_C={}
for each_year in list_year:
    sel_year_df=crime.loc[(crime['Apr - Mar reference period'] == each_year)]
    for each_sub in all_sub:
        final_df=sel_year_df.loc[(sel_year_df['Local Government Area'] == each_sub) & (sel_year_df['CSA Offence Division'] == 'C Drug offences')]
        total_C=final_df['Offence Count'].sum()
        name=str(each_year)+'_'+each_sub
        Year_sub_C[name]=total_C

In [53]:
#read the council file
councils = pd.read_csv('councils.txt', sep=':', header=None, error_bad_lines=False)
councils.head()

,0,1
0,BANYULE,"['Abbotsford', 'Airport West', 'Albert Park',..."
1,BRIMBANK,"['Altona', 'Altona North', 'Armadale', 'Ascot..."
2,DAREBIN,"['Ashburton', 'Ashwood', 'Avondale Heights', ..."
3,HOBSONS BAY,"['Balwyn', 'Balwyn North', 'Bentleigh', 'Bent..."
4,HUME,"['Box Hill', 'Braybrook', 'Brighton', 'Bright..."


In [54]:
#remove the space at the end for each council and make a council list
fin_coun=[]
coun = councils[0].tolist()
for council in coun:
    new = council.strip()
    fin_coun.append(new)
# use re to find corrisponding suburbs for each council and zip as a tuple list
all_suburb=[]
subu = councils[1]
for each in subu:
    ex_subu=re.findall(r'(?:\')(.*?)(?:\')', each)
    #make suburb in uppercase, for later use
    ex_subu=[x.upper() for x in ex_subu]
    all_suburb.append(ex_subu)
coun_subu=list(zip(fin_coun,all_suburb))
print(coun_subu)

[('BANYULE', ['ABBOTSFORD', 'AIRPORT WEST', 'ALBERT PARK', 'ALPHINGTON']), ('BRIMBANK', ['ALTONA', 'ALTONA NORTH', 'ARMADALE', 'ASCOT VALE']), ('DAREBIN', ['ASHBURTON', 'ASHWOOD', 'AVONDALE HEIGHTS', 'BALACLAVA']), ('HOBSONS BAY', ['BALWYN', 'BALWYN NORTH', 'BENTLEIGH', 'BENTLEIGH EAST']), ('HUME', ['BOX HILL', 'BRAYBROOK', 'BRIGHTON', 'BRIGHTON EAST']), ('MARIBYRNONG', ['BRUNSWICK', 'BRUNSWICK WEST', 'BULLEEN', 'BURWOOD']), ('MELBOURNE', ['CAMBERWELL', 'CANTERBURY', 'CARLTON NORTH', 'CARNEGIE']), ('MELTON', ['CAULFIELD', 'CAULFIELD NORTH', 'CAULFIELD SOUTH', 'CHADSTONE']), ('MOONEE VALLEY', ['CLIFTON HILL', 'COBURG', 'COBURG NORTH', 'COLLINGWOOD']), ('MORELAND', ['DONCASTER', 'EAGLEMONT', 'ELSTERNWICK', 'ELWOOD']), ('NILLUMBIK', ['ESSENDON', 'ESSENDON NORTH', 'FAIRFIELD', 'FITZROY']), ('WHITTLESEA', ['FITZROY NORTH', 'FLEMINGTON', 'FOOTSCRAY', 'GLEN IRIS']), ('WYNDHAM', ['GLENROY', 'GOWANBRAE', 'HADFIELD', 'HAMPTON']), ('YARRA', ['HAMPTON EAST', 'HAWTHORN', 'HEIDELBERG HEIGHTS', 'HEID

In [55]:
# suburb map with correct council area
correct_coun=[]
for i in range(len(sub)):
    #get suburb name
    each_sub=sub[i]
    for j in range(len(coun_subu)):
        co_su=coun_subu[j]
        #check whether suburb in the area list
        if each_sub not in co_su[1]:
            continue
        else:
            correct_coun.append(co_su[0])
len(correct_coun)

720

In [56]:
#calculate crime A average of past three years
A_ave=[]
for i in range(len(year)):
    #get three years
    year_1=int(year[i])-1
    year_2=int(year[i])-2
    year_3=int(year[i])-3
    #make up dict keys as year_council to find in dict
    name1=str(year_1)+'_'+correct_coun[i]
    name2=str(year_2)+'_'+correct_coun[i]
    name3=str(year_3)+'_'+correct_coun[i]
    v1=Year_sub_A[str(name1)]
    v2=Year_sub_A[str(name2)]
    v3=Year_sub_A[str(name3)]
    ave_3=(int(v1)+int(v2)+int(v3))/3
    A_ave.append(int(ave_3))
A_ave

[959,
 119,
 1923,
 824,
 2047,
 1245,
 610,
 925,
 1923,
 517,
 2047,
 71,
 684,
 958,
 1064,
 2532,
 142,
 2744,
 56,
 959,
 1150,
 2494,
 1173,
 278,
 330,
 268,
 288,
 2047,
 142,
 2489,
 84,
 1683,
 717,
 990,
 925,
 1508,
 3767,
 941,
 703,
 1412,
 824,
 885,
 3703,
 84,
 2524,
 958,
 334,
 824,
 2494,
 925,
 1566,
 824,
 1363,
 941,
 71,
 84,
 647,
 268,
 3703,
 1566,
 268,
 647,
 2744,
 824,
 824,
 56,
 3767,
 1923,
 3703,
 2744,
 1566,
 1253,
 278,
 1144,
 2403,
 1479,
 705,
 885,
 1245,
 717,
 1923,
 1202,
 824,
 330,
 2403,
 717,
 56,
 1923,
 235,
 101,
 2897,
 84,
 1071,
 1002,
 1173,
 1064,
 1064,
 3767,
 2006,
 313,
 1683,
 1363,
 925,
 1100,
 1683,
 1288,
 1245,
 1245,
 885,
 318,
 268,
 885,
 1540,
 925,
 330,
 659,
 1683,
 2494,
 958,
 334,
 278,
 1360,
 938,
 647,
 925,
 3767,
 1666,
 717,
 1471,
 1144,
 703,
 959,
 97,
 1083,
 941,
 659,
 2837,
 1479,
 663,
 1144,
 824,
 2532,
 1064,
 2403,
 2897,
 958,
 885,
 938,
 1683,
 1064,
 1071,
 659,
 1672,
 2744,
 136,
 2897

In [57]:
gr['crime_A_average']= A_ave

In [58]:
#calculate crime A average of past three years
B_ave=[]
for i in range(len(year)):
    year_1=int(year[i])-1
    year_2=int(year[i])-2
    year_3=int(year[i])-3
    name1=str(year_1)+'_'+correct_coun[i]
    name2=str(year_2)+'_'+correct_coun[i]
    name3=str(year_3)+'_'+correct_coun[i]
    v1=Year_sub_B[str(name1)]
    v2=Year_sub_B[str(name2)]
    v3=Year_sub_B[str(name3)]
    ave_3=(int(v1)+int(v2)+int(v3))/3
    B_ave.append(int(ave_3))
B_ave

[4476,
 452,
 8979,
 1890,
 9171,
 3886,
 1743,
 4701,
 8979,
 2973,
 9171,
 261,
 1695,
 5673,
 4200,
 5455,
 344,
 10289,
 98,
 4476,
 7827,
 11307,
 8523,
 489,
 1855,
 736,
 712,
 9171,
 344,
 11817,
 293,
 8938,
 1763,
 3070,
 4701,
 6073,
 18925,
 5813,
 1502,
 6364,
 1890,
 1901,
 18944,
 293,
 12864,
 5673,
 1407,
 1890,
 11307,
 4701,
 8895,
 1890,
 6287,
 5813,
 261,
 293,
 2861,
 736,
 18944,
 8895,
 736,
 2861,
 10289,
 1890,
 1890,
 98,
 18925,
 8979,
 18944,
 10289,
 8895,
 7401,
 489,
 6845,
 5212,
 4398,
 5479,
 1901,
 3886,
 1763,
 8979,
 5034,
 1890,
 1855,
 5212,
 1763,
 98,
 8979,
 506,
 251,
 11243,
 293,
 5783,
 4068,
 8523,
 4200,
 4200,
 18925,
 7754,
 545,
 8938,
 6287,
 4701,
 3636,
 8938,
 4242,
 3886,
 3886,
 1901,
 497,
 736,
 1901,
 6062,
 4701,
 1855,
 5219,
 8938,
 11307,
 5673,
 1407,
 489,
 6408,
 6892,
 2861,
 4701,
 18925,
 9333,
 1763,
 7759,
 6845,
 1502,
 4476,
 243,
 3612,
 5813,
 5219,
 9343,
 4398,
 4010,
 6845,
 1890,
 5455,
 4200,
 5212,
 112

In [59]:
gr['crime_B_average']= B_ave

In [60]:
#calculate crime A average of past three years
C_ave=[]
for i in range(len(year)):
    year_1=int(year[i])-1
    year_2=int(year[i])-2
    year_3=int(year[i])-3
    name1=str(year_1)+'_'+correct_coun[i]
    name2=str(year_2)+'_'+correct_coun[i]
    name3=str(year_3)+'_'+correct_coun[i]
    v1=Year_sub_C[str(name1)]
    v2=Year_sub_C[str(name2)]
    v3=Year_sub_C[str(name3)]
    ave_3=(int(v1)+int(v2)+int(v3))/3
    C_ave.append(int(ave_3))
C_ave

[324,
 22,
 620,
 159,
 742,
 447,
 228,
 308,
 620,
 288,
 742,
 20,
 161,
 628,
 432,
 713,
 39,
 780,
 19,
 324,
 922,
 1410,
 974,
 65,
 82,
 128,
 75,
 742,
 39,
 1268,
 23,
 509,
 318,
 411,
 308,
 553,
 2042,
 575,
 135,
 699,
 159,
 227,
 2039,
 23,
 738,
 628,
 88,
 159,
 1410,
 308,
 491,
 159,
 564,
 575,
 20,
 23,
 222,
 128,
 2039,
 491,
 128,
 222,
 780,
 159,
 159,
 19,
 2042,
 620,
 2039,
 780,
 491,
 484,
 65,
 375,
 586,
 502,
 335,
 227,
 447,
 318,
 620,
 333,
 159,
 82,
 586,
 318,
 19,
 620,
 74,
 18,
 942,
 23,
 419,
 328,
 974,
 432,
 432,
 2042,
 1051,
 65,
 509,
 564,
 308,
 457,
 509,
 480,
 447,
 447,
 227,
 140,
 128,
 227,
 477,
 308,
 82,
 338,
 509,
 1410,
 628,
 88,
 65,
 691,
 553,
 222,
 308,
 2042,
 523,
 318,
 779,
 375,
 135,
 324,
 33,
 368,
 575,
 338,
 1212,
 502,
 377,
 375,
 159,
 713,
 432,
 586,
 942,
 628,
 227,
 553,
 509,
 432,
 419,
 338,
 646,
 780,
 42,
 942,
 738,
 74,
 338,
 12,
 553,
 699,
 586,
 318,
 338,
 564,
 12,
 620,
 523,
 3

In [61]:
gr['crime_C_average']= C_ave
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,959,4476,324,not available,-1,-1,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,119,452,22,not available,-1,-1,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,1923,8979,620,not available,-1,-1,not available,-1,-1,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,824,1890,159,not available,-1,-1,not available,-1,-1,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,2047,9171,742,not available,-1,-1,not available,-1,-1,POINT (145.0889 -37.8555)


# Find Closest Primary School and Distance

Firstly, use `xml.etree.ElementTree` to get information from `schools.xml`. Create school name, type, latitude and longitude lists. Use `haversine`  to calculate the distance between two points, then use `find_clo_pri` function to return the school name and minimum distance.

The distance list store all the distance from one property to all schools, when we find the shortest distance, use index to find the corrisponding school name in the list.

In [62]:
# Get information from schools.xml
import xml.etree.ElementTree as et 
school_list=[]
xtree = et.parse("schools.xml")
xroot = xtree.getroot() 
# get school names
for child in xroot.iter('School_Name'):
    school_list.append(child.text)
print(len(school_list))
school_type=[]
# get school types
for child in xroot.iter('School_Type'):
    school_type.append(child.text)
print(len(school_type))

2254
2254


In [63]:
#get latitude and longitude
sch_x=[]
for child in xroot.iter('X'):
    sch_x.append(child.text)
print(len(sch_x))
sch_y=[]
for child in xroot.iter('Y'):
    sch_y.append(child.text)
print(len(sch_y))

2254
2254


In [64]:
#check the type for all schools
print(set(school_type))

{'Secondary', 'Language', 'Pri/Sec', 'Primary', 'Special'}


In [65]:
#seperate primary school and secondary school into two list
#also seperate their latitude and longitude
primary=[]
second=[]
pri_xy=[]
sec_xy=[]
for i in range(len(school_type)):
    if school_type[i]=='Pri/Sec':
        primary.append(school_list[i])
        pri_xy.append((sch_x[i],sch_y[i]))
        second.append(school_list[i])
        sec_xy.append((sch_x[i],sch_y[i]))
    elif school_type[i]=='Primary':
        primary.append(school_list[i])
        pri_xy.append((sch_x[i],sch_y[i]))
    elif school_type[i]=='Secondary':
        second.append(school_list[i])
        sec_xy.append((sch_x[i],sch_y[i]))
print(len(primary))
print(len(second))
print(len(pri_xy))
print(len(sec_xy))

1799
586
1799
586


In [66]:
# combine primary school name and their lan and lon
all_pri=[]
for i in range(len(primary)):
    tup=tuple([pri_xy[i],primary[i]])
    all_pri.append(tup)
print(len(all_pri))

1799


In [67]:
# input each house's lattitude and longitude
#return the school name and shortest distance
def find_clo_pri(housela, houselo, input_list):
    all_dis=[]
    for each in input_list:
        #school lan and lon
        Slon=float(each[0][0])
        Slan=float(each[0][1])
        each_dis=haversine(Slan,Slon,housela,houselo)
        all_dis.append(each_dis)
    #find minimum distance
    mini_dis=min(all_dis)
    #get index of minimum distance
    inde=all_dis.index(mini_dis)
    #find the school name accoding to index
    school_name=input_list[inde][1]
    sch_mini=(school_name,mini_dis)
    return sch_mini

In [68]:
Hlan=gr['Latitude']
Hlon=gr['Longitude']
close_pri=[]
# there are 720 properties
#for each property, use function get the school name and min distace
for i in range(len(Hlan)):
    Hou_lan=Hlan[i]
    Hou_lon=Hlon[i]
    result=find_clo_pri(Hou_lan,Hou_lon,all_pri)
    close_pri.append(result)
print(close_pri)

[("St Bede's School", 0.5034201888530943), ('Albanvale Primary School', 0.5264217365969028), ("St Brendan's School", 0.23289186253016436), ('Richmond West Primary School', 0.8774759378820088), ('Ashburton Primary School', 0.8782166739495798), ('Melbourne Grammar School', 0.8081408391689548), ('Footscray North Primary School', 0.22958831406272953), ('Balwyn North Primary School', 0.9340044938930978), ("St Monica's School", 0.4080470259553218), ("St Augustine's School", 1.036679973876135), ('Glen Iris Primary School', 0.7363626319140474), ('Moorabbin Primary School', 0.397018880050505), ('Bundoora Primary School', 0.4833160358303134), ('Brunswick North Primary School', 0.3848869434040784), ('Westbourne Grammar School', 0.2856123802765645), ('Bell Primary School', 0.6152009836091594), ('North Melbourne Primary School', 0.6347779337189629), ('Gardenvale Primary School', 0.3462722938712732), ('Highvale Primary School', 0.30829981669772305), ("St Paul's School", 0.5626213900139279), ('East P

In [69]:
# make tuple list into two lists, school name and distace to add in df
closest_primary_school=[]
distance_to_closest_primary=[]
for each_tuple in close_pri:
    closest_primary_school.append(each_tuple[0])
    distance_to_closest_primary.append(each_tuple[1])

In [70]:
gr['closest_primary_school']= closest_primary_school
gr['distance_to_closest_primary']= distance_to_closest_primary
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,959,4476,324,St Bede's School,0.503420,-1,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,119,452,22,Albanvale Primary School,0.526422,-1,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,1923,8979,620,St Brendan's School,0.232892,-1,not available,-1,-1,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,824,1890,159,Richmond West Primary School,0.877476,-1,not available,-1,-1,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,2047,9171,742,Ashburton Primary School,0.878217,-1,not available,-1,-1,POINT (145.0889 -37.8555)


# Find Primary School Ranking

use `urlopen` to open the website, and store as `primary.txt`.

Read the txt then use regular expression to find the school name with ranking. Create a dictionary, if the key has already appear there, no need to change the ranking, because the ranking number in order from 0-500.

Then we use `difflib.get_close_matches` to find primary school name with high similarities, `cutoff=0.9`. The cutoff uses higher probability because we want to keep the correctness of the data.

In [71]:
# get content from website save as txt
from urllib.request import urlopen
import re
html= urlopen("http://www.schoolcatchment.com.au/?p=12301")
content = html.read().decode('utf-8')
file = open('primary.txt','w',encoding='utf-8')
file.write(content)
file.close()

In [72]:
# filter out contents relates to school name and ranking use re
inputFile = open('primary.txt', 'r', encoding='utf-8').read()
pattern=inputFile.split("</tr>")
name_1 = re.findall('(?:<th class="column-1">)(.*?)(?:<\/th>)', pattern[0])
name_2 = re.findall('(?:<th class="column-2">)(.*?)(?:</th>)', pattern[0])
#delete the content do not need front and at the end
del pattern[0]
del pattern[500]
print(name_1)
print(name_2)
print(len(pattern))

['Ranking']
['School Name']
500


In [73]:
# create primary school ranking dictionary use re
pri_rank={}
for each_line in pattern:
    rank=re.findall('(?:<td class="column-1">)(.*?)(?:</td>)', each_line)
    name=re.findall('(?:<td class="column-2">)(.*?)(?:</td>)', each_line)
    if name[0] not in pri_rank.keys():
        pri_rank[name[0]]=rank[0]

print(pri_rank)

{"Presbyterian Ladies' College": '1', 'Haileybury College': '2', 'Ballarat Clarendon College': '3', 'Fitzroy Community School': '4', "Fintona Girls' School": '5', 'Huntingtower School': '6', 'Beverley Hills Primary School': '7', 'Oakleigh South Primary School': '8', 'Scotch College': '9', 'Serpell Primary School': '10', 'Camberwell Grammar School': '11', "Mentone Girls' Grammar School": '12', 'St Andrews Christian College': '13', 'Christ Church Grammar School': '14', 'Waverley Christian College': '15', 'Balwyn Primary School': '16', 'Erasmus School': '17', 'Lighthouse Christian College Cranbourne': '18', 'Glendal Primary School': '19', 'Thomastown East Primary School': '20', 'Doncaster Gardens Primary School': '21', "Korowa Anglican Girls' School": '22', 'Carey Baptist Grammar School': '23', 'Firbank Grammar School': '24', 'The Knox School': '25', 'Southmoor Primary School': '26', 'Loreto Mandeville Hall': '27', 'Wheelers Hill Primary School': '28', 'Camberwell Girls Grammar School': '

In [74]:
# use close school list to match the primary school name in ranking dict
import difflib
final_pri_rank=[]
for i in range(len(closest_primary_school)):
    # set n=1, return one match, butoff=0.9
    match=difflib.get_close_matches(closest_primary_school[i], pri_rank.keys(), n=1, cutoff=0.9)
    if match !=[]:
        final_pri_rank.append(pri_rank[match[0]])
    elif match==[]:
        final_pri_rank.append('not ranked')
    
final_pri_rank

['442',
 'not ranked',
 'not ranked',
 '452',
 '106',
 '79',
 '235',
 '142',
 '358',
 'not ranked',
 '120',
 '237',
 'not ranked',
 '448',
 '58',
 '385',
 '182',
 '138',
 '74',
 '211',
 'not ranked',
 '249',
 '62',
 '225',
 'not ranked',
 '346',
 '356',
 'not ranked',
 '285',
 '213',
 '411',
 '321',
 '235',
 '137',
 '211',
 '187',
 '82',
 'not ranked',
 '90',
 'not ranked',
 'not ranked',
 'not ranked',
 '69',
 '411',
 '107',
 'not ranked',
 '261',
 'not ranked',
 'not ranked',
 '380',
 '174',
 'not ranked',
 '265',
 '299',
 '182',
 '26',
 '384',
 '61',
 '69',
 '309',
 '346',
 'not ranked',
 'not ranked',
 'not ranked',
 '475',
 'not ranked',
 '89',
 '4',
 '290',
 '70',
 '174',
 '227',
 '219',
 '124',
 '393',
 'not ranked',
 'not ranked',
 '97',
 '44',
 '148',
 '399',
 'not ranked',
 'not ranked',
 'not ranked',
 '313',
 '278',
 '164',
 'not ranked',
 'not ranked',
 '66',
 '336',
 '66',
 '489',
 'not ranked',
 'not ranked',
 'not ranked',
 '238',
 '310',
 'not ranked',
 '182',
 'not ra

In [75]:
# there are 214 school cannot match with its name
final_pri_rank.count('not ranked')

214

In [76]:
gr['primary_school_ranking']= final_pri_rank
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,959,4476,324,St Bede's School,0.503420,442,not available,-1,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,119,452,22,Albanvale Primary School,0.526422,not ranked,not available,-1,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,1923,8979,620,St Brendan's School,0.232892,not ranked,not available,-1,-1,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,824,1890,159,Richmond West Primary School,0.877476,452,not available,-1,-1,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,2047,9171,742,Ashburton Primary School,0.878217,106,not available,-1,-1,POINT (145.0889 -37.8555)


# Find Closest Secondary School and Distance

Firstly, match all the school with lan and lon as tuple list.

This section is similar to primary school. Use `haversine`  to calculate the distance between two points, then use `find_clo_pri` function to return the school name and minimum distance.

The distance list store all the distance from one property to all schools, when we find the shortest distance, use index to find the corrisponding school name in the list.

In [77]:
# combine secondary school name and their lan and lon
all_sec=[]
for i in range(len(second)):
    tup=tuple([sec_xy[i],second[i]])
    all_sec.append(tup)
print(len(all_sec))

586


In [78]:
# there are 720 properties
#for each property, use function get the school name and min distace
close_sec=[]
for i in range(len(Hlan)):
    Hou_lan1=Hlan[i]
    Hou_lon1=Hlon[i]
    result=find_clo_pri(Hou_lan1,Hou_lon1,all_sec)
    close_sec.append(result)
print(close_sec)

[('Kew High School', 0.2065652131725788), ('Victoria University Secondary College', 1.600499913268575), ('Mount Alexander 7-12 College', 0.16085013970208964), ('Richmond High School', 0.5669401174734934), ('Ashwood High School', 1.617907438946183), ('Victorian College Of The Arts Secondary School', 0.5019886455326257), ('Footscray City College', 1.1418866044736702), ('Balwyn High School', 0.8027299813763065), ('Gilmore College For Girls', 0.7501929482706142), ('Frankston High School', 0.9718442458855205), ("Korowa Anglican Girls' School", 1.600880064999902), ('Our Lady of the Sacred Heart College', 1.934061636649668), ('Bundoora Secondary College', 0.5156091505750824), ('Brunswick Secondary College', 1.2467771613498033), ('Bayside P-12 College', 0.9132439948019949), ("St John's College Preston", 1.0707773018219835), ('University High School', 0.17995363960269312), ('Leibler Yavneh College', 1.2053663612492067), ('Highvale Secondary College', 0.4788026562493239), ('McKinnon Secondary Co

In [79]:
# make tuple list into two lists, school name and distace to add in df
closest_secondary_school=[]
distance_to_closest_secondary=[]
for each_tuple in close_sec:
    closest_secondary_school.append(each_tuple[0])
    distance_to_closest_secondary.append(each_tuple[1])

In [80]:
gr['closest_secondary_school']= closest_secondary_school
gr['distance_to_closest_secondary']= distance_to_closest_secondary
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,959,4476,324,St Bede's School,0.503420,442,Kew High School,0.206565,-1,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,119,452,22,Albanvale Primary School,0.526422,not ranked,Victoria University Secondary College,1.600500,-1,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,1923,8979,620,St Brendan's School,0.232892,not ranked,Mount Alexander 7-12 College,0.160850,-1,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,824,1890,159,Richmond West Primary School,0.877476,452,Richmond High School,0.566940,-1,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,2047,9171,742,Ashburton Primary School,0.878217,106,Ashwood High School,1.617907,-1,POINT (145.0889 -37.8555)


# Find Secondary School Ranking

Here we copied the content from the website, save as secondary.txt, then read from its content. Because there is okta login, we cannot use crawler to get content from website.

Read the txt then use regular expression to find the school name with ranking. Create a dictionary, if the key has already appear there, check the value of the ranking if old is smaller, keep the old ranking, because secondary is not in order.

Then we use `difflib.get_close_matches` to find secondary school name with high similarities, `cutoff=0.85`. There are few secondary school compare to primary school, therefore use 0.85 to find the name. 

In [ ]:
inputFile1 = '''
<!DOCTYPE html>
<html lang="en-US" class="no-js">
<head>
	<script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<!-- SchoolCatchment Sidebar -->
<ins class="adsbygoogle"
     style="display:block"
     data-ad-client="ca-pub-3991833096608295"
     data-ad-slot="1042533966"
     data-ad-format="auto"
     data-full-width-responsive="true"></ins>
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script>
<script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<script>
     (adsbygoogle = window.adsbygoogle || []).push({
          google_ad_client: "ca-pub-3991833096608295",
          enable_page_level_ads: true
     });
</script>
	<meta charset="UTF-8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<link rel="profile" href="http://gmpg.org/xfn/11">
		<link rel="pingback" href="http://www.schoolcatchment.com.au/xmlrpc.php">
		<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<title>Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government) &#8211; Australian Public School Information</title>
<link rel="dns-prefetch" href="//connect.facebook.net">
<link rel='dns-prefetch' href='//connect.facebook.net' />
<link rel='dns-prefetch' href='//fonts.googleapis.com' />
<link rel='dns-prefetch' href='//s.w.org' />
<link href='https://fonts.gstatic.com' crossorigin rel='preconnect' />
<link rel="alternate" type="application/rss+xml" title="Australian Public School Information &raquo; Feed" href="http://www.schoolcatchment.com.au/?feed=rss2" />
<link rel="alternate" type="application/rss+xml" title="Australian Public School Information &raquo; Comments Feed" href="http://www.schoolcatchment.com.au/?feed=comments-rss2" />
<link rel="alternate" type="application/rss+xml" title="Australian Public School Information &raquo; Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government) Comments Feed" href="http://www.schoolcatchment.com.au/?feed=rss2&#038;p=12301" />
<!-- This site uses the Google Analytics by MonsterInsights plugin v7.9.0 - Using Analytics tracking - https://www.monsterinsights.com/ -->

<script type="text/javascript" data-cfasync="false">

	var mi_version         = '7.9.0';

	var mi_track_user      = true;

	var mi_no_track_reason = '';

	

	var disableStr = 'ga-disable-UA-150427514-1';



	/* Function to detect opted out users */

	function __gaTrackerIsOptedOut() {

		return document.cookie.indexOf(disableStr + '=true') > -1;

	}



	/* Disable tracking if the opt-out cookie exists. */

	if ( __gaTrackerIsOptedOut() ) {

		window[disableStr] = true;

	}



	/* Opt-out function */

	function __gaTrackerOptout() {

	  document.cookie = disableStr + '=true; expires=Thu, 31 Dec 2099 23:59:59 UTC; path=/';

	  window[disableStr] = true;

	}

	

	if ( mi_track_user ) {

		(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){

			(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),

			m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)

		})(window,document,'script','//www.google-analytics.com/analytics.js','__gaTracker');



		__gaTracker('create', 'UA-150427514-1', 'auto');
		__gaTracker('set', 'forceSSL', true);
		__gaTracker('require', 'displayfeatures');
		__gaTracker('send','pageview');
	} else {

		console.log( "" );

		(function() {

			/* https://developers.google.com/analytics/devguides/collection/analyticsjs/ */

			var noopfn = function() {

				return null;

			};

			var noopnullfn = function() {

				return null;

			};

			var Tracker = function() {

				return null;

			};

			var p = Tracker.prototype;

			p.get = noopfn;

			p.set = noopfn;

			p.send = noopfn;

			var __gaTracker = function() {

				var len = arguments.length;

				if ( len === 0 ) {

					return;

				}

				var f = arguments[len-1];

				if ( typeof f !== 'object' || f === null || typeof f.hitCallback !== 'function' ) {

					console.log( 'Not running function __gaTracker(' + arguments[0] + " ....) because you are not being tracked. " + mi_no_track_reason );

					return;

				}

				try {

					f.hitCallback();

				} catch (ex) {



				}

			};

			__gaTracker.create = function() {

				return new Tracker();

			};

			__gaTracker.getByName = noopnullfn;

			__gaTracker.getAll = function() {

				return [];

			};

			__gaTracker.remove = noopfn;

			window['__gaTracker'] = __gaTracker;

					})();

		}

</script>

<!-- / Google Analytics by MonsterInsights -->

		<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/12.0.0-1\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/12.0.0-1\/svg\/","svgExt":".svg","source":{"concatemoji":"http:\/\/www.schoolcatchment.com.au\/wp-includes\/js\/wp-emoji-release.min.js?ver=5.2.4"}};
			!function(a,b,c){function d(a,b){var c=String.fromCharCode;l.clearRect(0,0,k.width,k.height),l.fillText(c.apply(this,a),0,0);var d=k.toDataURL();l.clearRect(0,0,k.width,k.height),l.fillText(c.apply(this,b),0,0);var e=k.toDataURL();return d===e}function e(a){var b;if(!l||!l.fillText)return!1;switch(l.textBaseline="top",l.font="600 32px Arial",a){case"flag":return!(b=d([55356,56826,55356,56819],[55356,56826,8203,55356,56819]))&&(b=d([55356,57332,56128,56423,56128,56418,56128,56421,56128,56430,56128,56423,56128,56447],[55356,57332,8203,56128,56423,8203,56128,56418,8203,56128,56421,8203,56128,56430,8203,56128,56423,8203,56128,56447]),!b);case"emoji":return b=d([55357,56424,55356,57342,8205,55358,56605,8205,55357,56424,55356,57340],[55357,56424,55356,57342,8203,55358,56605,8203,55357,56424,55356,57340]),!b}return!1}function f(a){var c=b.createElement("script");c.src=a,c.defer=c.type="text/javascript",b.getElementsByTagName("head")[0].appendChild(c)}var g,h,i,j,k=b.createElement("canvas"),l=k.getContext&&k.getContext("2d");for(j=Array("flag","emoji"),c.supports={everything:!0,everythingExceptFlag:!0},i=0;i<j.length;i++)c.supports[j[i]]=e(j[i]),c.supports.everything=c.supports.everything&&c.supports[j[i]],"flag"!==j[i]&&(c.supports.everythingExceptFlag=c.supports.everythingExceptFlag&&c.supports[j[i]]);c.supports.everythingExceptFlag=c.supports.everythingExceptFlag&&!c.supports.flag,c.DOMReady=!1,c.readyCallback=function(){c.DOMReady=!0},c.supports.everything||(h=function(){c.readyCallback()},b.addEventListener?(b.addEventListener("DOMContentLoaded",h,!1),a.addEventListener("load",h,!1)):(a.attachEvent("onload",h),b.attachEvent("onreadystatechange",function(){"complete"===b.readyState&&c.readyCallback()})),g=c.source||{},g.concatemoji?f(g.concatemoji):g.wpemoji&&g.twemoji&&(f(g.twemoji),f(g.wpemoji)))}(window,document,window._wpemojiSettings);
		</script>
		<!-- managing ads with Advanced Ads – https://wpadvancedads.com/ --><script>
					advanced_ads_ready=function(){var fns=[],listener,doc=typeof document==="object"&&document,hack=doc&&doc.documentElement.doScroll,domContentLoaded="DOMContentLoaded",loaded=doc&&(hack?/^loaded|^c/:/^loaded|^i|^c/).test(doc.readyState);if(!loaded&&doc){listener=function(){doc.removeEventListener(domContentLoaded,listener);window.removeEventListener("load",listener);loaded=1;while(listener=fns.shift())listener()};doc.addEventListener(domContentLoaded,listener);window.addEventListener("load",listener)}return function(fn){loaded?setTimeout(fn,0):fns.push(fn)}}();
			</script><style type="text/css">
img.wp-smiley,
img.emoji {
	display: inline !important;
	border: none !important;
	box-shadow: none !important;
	height: 1em !important;
	width: 1em !important;
	margin: 0 .07em !important;
	vertical-align: -0.1em !important;
	background: none !important;
	padding: 0 !important;
}
</style>
	<link rel='stylesheet' id='wp-block-library-css'  href='http://www.schoolcatchment.com.au/wp-includes/css/dist/block-library/style.min.css?ver=5.2.4' type='text/css' media='all' />
<link rel='stylesheet' id='wp-block-library-theme-css'  href='http://www.schoolcatchment.com.au/wp-includes/css/dist/block-library/theme.min.css?ver=5.2.4' type='text/css' media='all' />
<link rel='stylesheet' id='wp-email-css'  href='http://www.schoolcatchment.com.au/wp-content/plugins/wp-email/email-css.css?ver=2.67.6' type='text/css' media='all' />
<link rel='stylesheet' id='twentysixteen-fonts-css'  href='https://fonts.googleapis.com/css?family=Merriweather%3A400%2C700%2C900%2C400italic%2C700italic%2C900italic%7CMontserrat%3A400%2C700%7CInconsolata%3A400&#038;subset=latin%2Clatin-ext' type='text/css' media='all' />
<link rel='stylesheet' id='genericons-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/genericons/genericons.css?ver=3.4.1' type='text/css' media='all' />
<link rel='stylesheet' id='twentysixteen-style-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/style.css?ver=5.2.4' type='text/css' media='all' />
<link rel='stylesheet' id='twentysixteen-block-style-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/css/blocks.css?ver=20181230' type='text/css' media='all' />
<!--[if lt IE 10]>
<link rel='stylesheet' id='twentysixteen-ie-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/css/ie.css?ver=20160816' type='text/css' media='all' />
<![endif]-->
<!--[if lt IE 9]>
<link rel='stylesheet' id='twentysixteen-ie8-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/css/ie8.css?ver=20160816' type='text/css' media='all' />
<![endif]-->
<!--[if lt IE 8]>
<link rel='stylesheet' id='twentysixteen-ie7-css'  href='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/css/ie7.css?ver=20160816' type='text/css' media='all' />
<![endif]-->
<link rel='stylesheet' id='tablepress-responsive-tables-css'  href='http://www.schoolcatchment.com.au/wp-content/plugins/tablepress-responsive-tables/css/responsive.dataTables.min.css?ver=1.5' type='text/css' media='all' />
<link rel='stylesheet' id='tablepress-default-css'  href='http://www.schoolcatchment.com.au/wp-content/plugins/tablepress/css/default.min.css?ver=1.9.2' type='text/css' media='all' />
<!--[if !IE]><!-->
<link rel='stylesheet' id='tablepress-responsive-tables-flip-css'  href='http://www.schoolcatchment.com.au/wp-content/plugins/tablepress-responsive-tables/css/tablepress-responsive-flip.min.css?ver=1.5' type='text/css' media='all' />
<!--<![endif]-->
<script type='text/javascript'>
/* <![CDATA[ */
var monsterinsights_frontend = {"js_events_tracking":"true","download_extensions":"doc,exe,js,pdf,ppt,tgz,zip,xls","inbound_paths":"[{\"path\":\"\\\/go\\\/\",\"label\":\"affiliate\"},{\"path\":\"\\\/recommend\\\/\",\"label\":\"affiliate\"}]","home_url":"http:\/\/www.schoolcatchment.com.au","hash_tracking":"false"};
/* ]]> */
</script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/plugins/google-analytics-for-wordpress/assets/js/frontend.min.js?ver=7.9.0'></script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-includes/js/jquery/jquery.js?ver=1.12.4-wp'></script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-includes/js/jquery/jquery-migrate.min.js?ver=1.4.1'></script>
<!--[if lt IE 9]>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/js/html5.js?ver=3.7.3'></script>
<![endif]-->
<link rel='https://api.w.org/' href='http://www.schoolcatchment.com.au/index.php?rest_route=/' />
<link rel="EditURI" type="application/rsd+xml" title="RSD" href="http://www.schoolcatchment.com.au/xmlrpc.php?rsd" />
<link rel="wlwmanifest" type="application/wlwmanifest+xml" href="http://www.schoolcatchment.com.au/wp-includes/wlwmanifest.xml" /> 
<link rel='prev' title='Killara Public School In-Person' href='http://www.schoolcatchment.com.au/?p=12284' />
<link rel='next' title='Interesting Tidbit With Killara Public School' href='http://www.schoolcatchment.com.au/?p=12307' />
<meta name="generator" content="WordPress 5.2.4" />
<link rel="canonical" href="http://www.schoolcatchment.com.au/?p=12301" />
<link rel='shortlink' href='http://www.schoolcatchment.com.au/?p=12301' />
<link rel="alternate" type="application/json+oembed" href="http://www.schoolcatchment.com.au/index.php?rest_route=%2Foembed%2F1.0%2Fembed&#038;url=http%3A%2F%2Fwww.schoolcatchment.com.au%2F%3Fp%3D12301" />
<link rel="alternate" type="text/xml+oembed" href="http://www.schoolcatchment.com.au/index.php?rest_route=%2Foembed%2F1.0%2Fembed&#038;url=http%3A%2F%2Fwww.schoolcatchment.com.au%2F%3Fp%3D12301&#038;format=xml" />
<script type="text/javascript">
(function(url){
	if(/(?:Chrome\/26\.0\.1410\.63 Safari\/537\.31|WordfenceTestMonBot)/.test(navigator.userAgent)){ return; }
	var addEvent = function(evt, handler) {
		if (window.addEventListener) {
			document.addEventListener(evt, handler, false);
		} else if (window.attachEvent) {
			document.attachEvent('on' + evt, handler);
		}
	};
	var removeEvent = function(evt, handler) {
		if (window.removeEventListener) {
			document.removeEventListener(evt, handler, false);
		} else if (window.detachEvent) {
			document.detachEvent('on' + evt, handler);
		}
	};
	var evts = 'contextmenu dblclick drag dragend dragenter dragleave dragover dragstart drop keydown keypress keyup mousedown mousemove mouseout mouseover mouseup mousewheel scroll'.split(' ');
	var logHuman = function() {
		if (window.wfLogHumanRan) { return; }
		window.wfLogHumanRan = true;
		var wfscr = document.createElement('script');
		wfscr.type = 'text/javascript';
		wfscr.async = true;
		wfscr.src = url + '&r=' + Math.random();
		(document.getElementsByTagName('head')[0]||document.getElementsByTagName('body')[0]).appendChild(wfscr);
		for (var i = 0; i < evts.length; i++) {
			removeEvent(evts[i], logHuman);
		}
	};
	for (var i = 0; i < evts.length; i++) {
		addEvent(evts[i], logHuman);
	}
})('//www.schoolcatchment.com.au/?wordfence_lh=1&hid=C18914BC96D1D1F1596F1B9A44C7A506');
</script><!--[if !IE]><!-->
<!--<![endif]-->
		<style type="text/css">.recentcomments a{display:inline !important;padding:0 !important;margin:0 !important;}</style>
		<meta property="og:site_name" content="Australian Public School Information">
<meta property="og:type" content="article">
<meta property="og:locale" content="en_US">
<meta property="og:url" content="http://www.schoolcatchment.com.au/?p=12301">
<meta property="og:title" content="Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government)">
<meta property="og:description" content="This is a special entry for Lyn. Top ranking Victoria Melbourne primary schools for the year of 2017. The result is a combination of both year 3 and 5 numbers from NAPLAN. One thing to note is some schools listed have very low enrolment number so the result is possibly not a reliable reflection of&hellip;">
<meta property="article:published_time" content="2019-09-27T13:42:37+00:00">
<meta property="article:modified_time" content="2019-09-27T13:42:39+00:00">
<meta property="article:author" content="http://www.schoolcatchment.com.au/?author=1">
<meta property="article:section" content="General Education">
<meta property="article:tag" content="Victoria Melbourne Education">
<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<script>

     (adsbygoogle = window.adsbygoogle || []).push({

          google_ad_client: "ca-pub-3991833096608295",

          enable_page_level_ads: true

     });

</script><meta name="twitter:partner" content="tfwp"><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
<script>(adsbygoogle = window.adsbygoogle || []).push({
google_ad_client: "ca-pub-3991833096608295",
enable_page_level_ads: true,
});
</script>

<meta name="twitter:card" content="summary"><meta name="twitter:title" content="Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government)"><meta name="twitter:description" content="This is a special entry for Lyn. Top ranking Victoria Melbourne primary schools for the year of 2017. The result is a combination of both year 3 and 5 numbers from NAPLAN. One thing to note is some schools listed have very low enrolment number so the result is possibly not a reliable reflection of&hellip;">
</head>

<body class="post-template-default single single-post postid-12301 single-format-standard wp-embed-responsive">

<div id="page" class="site">
	<div class="site-inner">
		<a class="skip-link screen-reader-text" href="#content">Skip to content</a>

		<header id="masthead" class="site-header" role="banner">
			<div class="site-header-main">
				<div class="site-branding">
					
											<p class="site-title"><a href="http://www.schoolcatchment.com.au/" rel="home">Australian Public School Information</a></p>
												<p class="site-description">All you need to know about Australian Public School, including catchment/zone/boundary information.</p>
									</div><!-- .site-branding -->

									<button id="menu-toggle" class="menu-toggle">Menu</button>

					<div id="site-header-menu" class="site-header-menu">
													<nav id="site-navigation" class="main-navigation" role="navigation" aria-label="Primary Menu">
								<div class="menu-main-menu-container"><ul id="menu-main-menu" class="primary-menu"><li id="menu-item-11854" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11854"><a href="http://www.schoolcatchment.com.au/?page_id=5985">Top Ranking Australian Schools</a></li>
<li id="menu-item-11855" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11855"><a href="http://www.schoolcatchment.com.au/?page_id=5654">Queensland School Catchment Map</a></li>
<li id="menu-item-11856" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11856"><a href="http://www.schoolcatchment.com.au/?page_id=5367">South Australia Public School Zone Map</a></li>
<li id="menu-item-11857" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11857"><a href="http://www.schoolcatchment.com.au/?page_id=1148">Western Australia School Local Intake Area (Catchment) Map</a></li>
<li id="menu-item-11858" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11858"><a href="http://www.schoolcatchment.com.au/?page_id=1051">ACT Public School Priority Enrolment Area (Catchment) Map</a></li>
<li id="menu-item-11859" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11859"><a href="http://www.schoolcatchment.com.au/?page_id=825">Victoria/Melbourne Public School Zone Map</a></li>
<li id="menu-item-11860" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11860"><a href="http://www.schoolcatchment.com.au/?page_id=488">Sydney High School Catchment Map</a></li>
<li id="menu-item-11861" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11861"><a href="http://www.schoolcatchment.com.au/?page_id=36">NSW/Sydney Public School Catchment Map</a></li>
<li id="menu-item-11862" class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11862"><a href="http://www.schoolcatchment.com.au/?page_id=2">About us</a></li>
</ul></div>							</nav><!-- .main-navigation -->
						
											</div><!-- .site-header-menu -->
							</div><!-- .site-header-main -->

					</header><!-- .site-header -->

		<div id="content" class="site-content">

<div id="primary" class="content-area">
	<main id="main" class="site-main" role="main">
		
<article id="post-12301" class="post-12301 post type-post status-publish format-standard hentry category-general-education category-victoria-melbourne-education">
	<header class="entry-header">
		<h1 class="entry-title">Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government)</h1>	</header><!-- .entry-header -->

	
	
	<div class="entry-content">
		<p>This is a special entry for Lyn.</p>
<p>Top ranking Victoria Melbourne primary schools for the year of 2017. The result is a combination of both year 3 and 5 numbers from NAPLAN. One thing to note is some schools listed have very low enrolment number so the result is possibly not a reliable reflection of the entire school’s performance and historically can have big fluctuations as well. This is why I also included the enrolment number here, some school got big enrolment number as well due to they are classified as combined with both primary and secondary components counted together. One more thing to note is that percentage is based on the number one primary school regardless of school sector Australia wide.</p>

<table id="tablepress-233" class="tablepress tablepress-id-233 tablepress-responsive-phone">
<thead>
<tr class="row-1 odd">
	<th class="column-1">Ranking</th><th class="column-2">School Name</th><th class="column-3">State</th><th class="column-4">School Sector</th><th class="column-5">Location</th><th class="column-6">Total Enrolment</th><th class="column-7">Combined Score As Percentage of Number One School In Australia Regardless of School Sector</th>
</tr>
</thead>
<tbody class="row-hover">
<tr class="row-2 even">
	<td class="column-1">1</td><td class="column-2">Presbyterian Ladies' College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1412</td><td class="column-7">96.65%</td>
</tr>
<tr class="row-3 odd">
	<td class="column-1">2</td><td class="column-2">Haileybury College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">3754</td><td class="column-7">95.69%</td>
</tr>
<tr class="row-4 even">
	<td class="column-1">3</td><td class="column-2">Ballarat Clarendon College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">1326</td><td class="column-7">93.29%</td>
</tr>
<tr class="row-5 odd">
	<td class="column-1">4</td><td class="column-2">Fitzroy Community School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">113</td><td class="column-7">93.13%</td>
</tr>
<tr class="row-6 even">
	<td class="column-1">5</td><td class="column-2">Fintona Girls' School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">456</td><td class="column-7">92.96%</td>
</tr>
<tr class="row-7 odd">
	<td class="column-1">6</td><td class="column-2">Huntingtower School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">686</td><td class="column-7">92.88%</td>
</tr>
<tr class="row-8 even">
	<td class="column-1">7</td><td class="column-2">Beverley Hills Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">473</td><td class="column-7">92.74%</td>
</tr>
<tr class="row-9 odd">
	<td class="column-1">8</td><td class="column-2">Oakleigh South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1006</td><td class="column-7">91.90%</td>
</tr>
<tr class="row-10 even">
	<td class="column-1">9</td><td class="column-2">Scotch College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1883</td><td class="column-7">91.85%</td>
</tr>
<tr class="row-11 odd">
	<td class="column-1">10</td><td class="column-2">Serpell Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1159</td><td class="column-7">91.81%</td>
</tr>
<tr class="row-12 even">
	<td class="column-1">11</td><td class="column-2">Camberwell Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1301</td><td class="column-7">91.69%</td>
</tr>
<tr class="row-13 odd">
	<td class="column-1">12</td><td class="column-2">Mentone Girls' Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">739</td><td class="column-7">91.40%</td>
</tr>
<tr class="row-14 even">
	<td class="column-1">13</td><td class="column-2">St Andrews Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">581</td><td class="column-7">91.25%</td>
</tr>
<tr class="row-15 odd">
	<td class="column-1">14</td><td class="column-2">Christ Church Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">292</td><td class="column-7">90.89%</td>
</tr>
<tr class="row-16 even">
	<td class="column-1">15</td><td class="column-2">Waverley Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1867</td><td class="column-7">90.65%</td>
</tr>
<tr class="row-17 odd">
	<td class="column-1">16</td><td class="column-2">Balwyn Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">537</td><td class="column-7">90.56%</td>
</tr>
<tr class="row-18 even">
	<td class="column-1">17</td><td class="column-2">Erasmus School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">106</td><td class="column-7">90.32%</td>
</tr>
<tr class="row-19 odd">
	<td class="column-1">18</td><td class="column-2">Lighthouse Christian College Cranbourne</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">863</td><td class="column-7">90.28%</td>
</tr>
<tr class="row-20 even">
	<td class="column-1">19</td><td class="column-2">Glendal Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">873</td><td class="column-7">90.15%</td>
</tr>
<tr class="row-21 odd">
	<td class="column-1">20</td><td class="column-2">Thomastown East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">169</td><td class="column-7">90.13%</td>
</tr>
<tr class="row-22 even">
	<td class="column-1">21</td><td class="column-2">Doncaster Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">681</td><td class="column-7">90.03%</td>
</tr>
<tr class="row-23 odd">
	<td class="column-1">22</td><td class="column-2">Korowa Anglican Girls' School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">620</td><td class="column-7">90.03%</td>
</tr>
<tr class="row-24 even">
	<td class="column-1">23</td><td class="column-2">Carey Baptist Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2248</td><td class="column-7">90.01%</td>
</tr>
<tr class="row-25 odd">
	<td class="column-1">24</td><td class="column-2">Firbank Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1107</td><td class="column-7">89.99%</td>
</tr>
<tr class="row-26 even">
	<td class="column-1">25</td><td class="column-2">The Knox School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">567</td><td class="column-7">89.99%</td>
</tr>
<tr class="row-27 odd">
	<td class="column-1">26</td><td class="column-2">Southmoor Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">432</td><td class="column-7">89.84%</td>
</tr>
<tr class="row-28 even">
	<td class="column-1">27</td><td class="column-2">Loreto Mandeville Hall</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">1167</td><td class="column-7">89.77%</td>
</tr>
<tr class="row-29 odd">
	<td class="column-1">28</td><td class="column-2">Wheelers Hill Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">528</td><td class="column-7">89.74%</td>
</tr>
<tr class="row-30 even">
	<td class="column-1">29</td><td class="column-2">Camberwell Girls Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">712</td><td class="column-7">89.56%</td>
</tr>
<tr class="row-31 odd">
	<td class="column-1">30</td><td class="column-2">Lauriston Girls' School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">897</td><td class="column-7">89.50%</td>
</tr>
<tr class="row-32 even">
	<td class="column-1">31</td><td class="column-2">Bentleigh West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">605</td><td class="column-7">89.44%</td>
</tr>
<tr class="row-33 odd">
	<td class="column-1">32</td><td class="column-2">Holy Trinity Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">381</td><td class="column-7">89.44%</td>
</tr>
<tr class="row-34 even">
	<td class="column-1">33</td><td class="column-2">St Leonard's College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1426</td><td class="column-7">89.41%</td>
</tr>
<tr class="row-35 odd">
	<td class="column-1">34</td><td class="column-2">Sholem Aleichem College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">180</td><td class="column-7">89.31%</td>
</tr>
<tr class="row-36 even">
	<td class="column-1">35</td><td class="column-2">Camberwell Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">654</td><td class="column-7">89.12%</td>
</tr>
<tr class="row-37 odd">
	<td class="column-1">36</td><td class="column-2">Camelot Rise Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">444</td><td class="column-7">89.10%</td>
</tr>
<tr class="row-38 even">
	<td class="column-1">37</td><td class="column-2">Knox Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">292</td><td class="column-7">89.10%</td>
</tr>
<tr class="row-39 odd">
	<td class="column-1">38</td><td class="column-2">Ivanhoe Girls' Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">920</td><td class="column-7">88.93%</td>
</tr>
<tr class="row-40 even">
	<td class="column-1">39</td><td class="column-2">Burwood East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">274</td><td class="column-7">88.88%</td>
</tr>
<tr class="row-41 odd">
	<td class="column-1">40</td><td class="column-2">Genazzano FCJ College</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">1026</td><td class="column-7">88.88%</td>
</tr>
<tr class="row-42 even">
	<td class="column-1">41</td><td class="column-2">Methodist Ladies' College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2034</td><td class="column-7">88.81%</td>
</tr>
<tr class="row-43 odd">
	<td class="column-1">42</td><td class="column-2">Deepdene Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">452</td><td class="column-7">88.79%</td>
</tr>
<tr class="row-44 even">
	<td class="column-1">43</td><td class="column-2">Canterbury Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">605</td><td class="column-7">88.71%</td>
</tr>
<tr class="row-45 odd">
	<td class="column-1">44</td><td class="column-2">South Yarra Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">377</td><td class="column-7">88.52%</td>
</tr>
<tr class="row-46 even">
	<td class="column-1">45</td><td class="column-2">Mount View Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1083</td><td class="column-7">88.41%</td>
</tr>
<tr class="row-47 odd">
	<td class="column-1">46</td><td class="column-2">Glen Waverley Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">864</td><td class="column-7">88.41%</td>
</tr>
<tr class="row-48 even">
	<td class="column-1">47</td><td class="column-2">Vermont Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">694</td><td class="column-7">88.40%</td>
</tr>
<tr class="row-49 odd">
	<td class="column-1">48</td><td class="column-2">Eltham East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">647</td><td class="column-7">88.35%</td>
</tr>
<tr class="row-50 even">
	<td class="column-1">49</td><td class="column-2">Good Shepherd School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">488</td><td class="column-7">88.35%</td>
</tr>
<tr class="row-51 odd">
	<td class="column-1">50</td><td class="column-2">St Cecilia's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">253</td><td class="column-7">88.33%</td>
</tr>
<tr class="row-52 even">
	<td class="column-1">51</td><td class="column-2">Pinewood Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">656</td><td class="column-7">88.26%</td>
</tr>
<tr class="row-53 odd">
	<td class="column-1">52</td><td class="column-2">Preston West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">678</td><td class="column-7">88.23%</td>
</tr>
<tr class="row-54 even">
	<td class="column-1">53</td><td class="column-2">Kingswood College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">567</td><td class="column-7">88.23%</td>
</tr>
<tr class="row-55 odd">
	<td class="column-1">54</td><td class="column-2">Ruyton Girls' School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">839</td><td class="column-7">88.17%</td>
</tr>
<tr class="row-56 even">
	<td class="column-1">55</td><td class="column-2">Toorak College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">737</td><td class="column-7">88.17%</td>
</tr>
<tr class="row-57 odd">
	<td class="column-1">56</td><td class="column-2">Trinity Grammar School Kew</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1381</td><td class="column-7">88.05%</td>
</tr>
<tr class="row-58 even">
	<td class="column-1">57</td><td class="column-2">Good Shepherd Lutheran Primary School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">529</td><td class="column-7">88.04%</td>
</tr>
<tr class="row-59 odd">
	<td class="column-1">58</td><td class="column-2">Westbourne Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1479</td><td class="column-7">88.02%</td>
</tr>
<tr class="row-60 even">
	<td class="column-1">59</td><td class="column-2">St Margaret's School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">824</td><td class="column-7">87.97%</td>
</tr>
<tr class="row-61 odd">
	<td class="column-1">60</td><td class="column-2">St Kevin's College</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">2037</td><td class="column-7">87.93%</td>
</tr>
<tr class="row-62 even">
	<td class="column-1">61</td><td class="column-2">Kilvington Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">723</td><td class="column-7">87.90%</td>
</tr>
<tr class="row-63 odd">
	<td class="column-1">62</td><td class="column-2">Glenferrie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">357</td><td class="column-7">87.83%</td>
</tr>
<tr class="row-64 even">
	<td class="column-1">63</td><td class="column-2">Livingstone Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">774</td><td class="column-7">87.80%</td>
</tr>
<tr class="row-65 odd">
	<td class="column-1">64</td><td class="column-2">Templeton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">720</td><td class="column-7">87.80%</td>
</tr>
<tr class="row-66 even">
	<td class="column-1">65</td><td class="column-2">Melbourne Girls Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">946</td><td class="column-7">87.76%</td>
</tr>
<tr class="row-67 odd">
	<td class="column-1">66</td><td class="column-2">Sacred Heart School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">277</td><td class="column-7">87.61%</td>
</tr>
<tr class="row-68 even">
	<td class="column-1">67</td><td class="column-2">Our Lady of Lourdes School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">140</td><td class="column-7">87.59%</td>
</tr>
<tr class="row-69 odd">
	<td class="column-1">68</td><td class="column-2">Malvern Central School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">416</td><td class="column-7">87.57%</td>
</tr>
<tr class="row-70 even">
	<td class="column-1">69</td><td class="column-2">Strathcona Baptist Girls' Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">733</td><td class="column-7">87.52%</td>
</tr>
<tr class="row-71 odd">
	<td class="column-1">70</td><td class="column-2">Brighton Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1218</td><td class="column-7">87.52%</td>
</tr>
<tr class="row-72 even">
	<td class="column-1">71</td><td class="column-2">Birralee Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">426</td><td class="column-7">87.50%</td>
</tr>
<tr class="row-73 odd">
	<td class="column-1">72</td><td class="column-2">Chilwell Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">374</td><td class="column-7">87.49%</td>
</tr>
<tr class="row-74 even">
	<td class="column-1">73</td><td class="column-2">St Kilda Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">350</td><td class="column-7">87.45%</td>
</tr>
<tr class="row-75 odd">
	<td class="column-1">74</td><td class="column-2">Highvale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">294</td><td class="column-7">87.40%</td>
</tr>
<tr class="row-76 even">
	<td class="column-1">75</td><td class="column-2">Melbourne Montessori School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">266</td><td class="column-7">87.38%</td>
</tr>
<tr class="row-77 odd">
	<td class="column-1">76</td><td class="column-2">Hampton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">731</td><td class="column-7">87.33%</td>
</tr>
<tr class="row-78 even">
	<td class="column-1">77</td><td class="column-2">Carnegie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">590</td><td class="column-7">87.30%</td>
</tr>
<tr class="row-79 odd">
	<td class="column-1">78</td><td class="column-2">Hartwell Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">710</td><td class="column-7">87.30%</td>
</tr>
<tr class="row-80 even">
	<td class="column-1">79</td><td class="column-2">Melbourne Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1794</td><td class="column-7">87.30%</td>
</tr>
<tr class="row-81 odd">
	<td class="column-1">80</td><td class="column-2">Murrumbeena Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">559</td><td class="column-7">87.28%</td>
</tr>
<tr class="row-82 even">
	<td class="column-1">81</td><td class="column-2">Alphington Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">428</td><td class="column-7">87.23%</td>
</tr>
<tr class="row-83 odd">
	<td class="column-1">82</td><td class="column-2">St Dominic's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">286</td><td class="column-7">87.20%</td>
</tr>
<tr class="row-84 even">
	<td class="column-1">83</td><td class="column-2">Toorak Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">503</td><td class="column-7">87.18%</td>
</tr>
<tr class="row-85 odd">
	<td class="column-1">84</td><td class="column-2">Sandridge School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">112</td><td class="column-7">87.18%</td>
</tr>
<tr class="row-86 even">
	<td class="column-1">85</td><td class="column-2">Brandon Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">390</td><td class="column-7">87.14%</td>
</tr>
<tr class="row-87 odd">
	<td class="column-1">86</td><td class="column-2">Banyule Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">654</td><td class="column-7">87.13%</td>
</tr>
<tr class="row-88 even">
	<td class="column-1">87</td><td class="column-2">Armadale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">370</td><td class="column-7">87.08%</td>
</tr>
<tr class="row-89 odd">
	<td class="column-1">88</td><td class="column-2">Park Orchards Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">323</td><td class="column-7">87.04%</td>
</tr>
<tr class="row-90 even">
	<td class="column-1">89</td><td class="column-2">Wattle Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">391</td><td class="column-7">87.02%</td>
</tr>
<tr class="row-91 odd">
	<td class="column-1">90</td><td class="column-2">Our Lady Help of Christians School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">200</td><td class="column-7">86.99%</td>
</tr>
<tr class="row-92 even">
	<td class="column-1">91</td><td class="column-2">Kerrimuir Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">344</td><td class="column-7">86.96%</td>
</tr>
<tr class="row-93 odd">
	<td class="column-1">92</td><td class="column-2">Caulfield Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">3082</td><td class="column-7">86.96%</td>
</tr>
<tr class="row-94 even">
	<td class="column-1">93</td><td class="column-2">Greythorn Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">565</td><td class="column-7">86.94%</td>
</tr>
<tr class="row-95 odd">
	<td class="column-1">94</td><td class="column-2">Casey Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">807</td><td class="column-7">86.89%</td>
</tr>
<tr class="row-96 even">
	<td class="column-1">95</td><td class="column-2">St Joan of Arc School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">375</td><td class="column-7">86.80%</td>
</tr>
<tr class="row-97 odd">
	<td class="column-1">96</td><td class="column-2">Mount Waverley Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">183</td><td class="column-7">86.63%</td>
</tr>
<tr class="row-98 even">
	<td class="column-1">97</td><td class="column-2">Footscray City Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">365</td><td class="column-7">86.61%</td>
</tr>
<tr class="row-99 odd">
	<td class="column-1">98</td><td class="column-2">Burwood Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">341</td><td class="column-7">86.61%</td>
</tr>
<tr class="row-100 even">
	<td class="column-1">99</td><td class="column-2">Caulfield Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">271</td><td class="column-7">86.61%</td>
</tr>
<tr class="row-101 odd">
	<td class="column-1">100</td><td class="column-2">Ivanhoe Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">615</td><td class="column-7">86.58%</td>
</tr>
<tr class="row-102 even">
	<td class="column-1">101</td><td class="column-2">Ss Peter &amp; Paul's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">162</td><td class="column-7">86.58%</td>
</tr>
<tr class="row-103 odd">
	<td class="column-1">102</td><td class="column-2">Paynesville Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">153</td><td class="column-7">86.56%</td>
</tr>
<tr class="row-104 even">
	<td class="column-1">103</td><td class="column-2">Fitzroy North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">532</td><td class="column-7">86.54%</td>
</tr>
<tr class="row-105 odd">
	<td class="column-1">104</td><td class="column-2">Black Rock Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">586</td><td class="column-7">86.51%</td>
</tr>
<tr class="row-106 even">
	<td class="column-1">105</td><td class="column-2">Yarraville West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">821</td><td class="column-7">86.49%</td>
</tr>
<tr class="row-107 odd">
	<td class="column-1">106</td><td class="column-2">Ashburton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">551</td><td class="column-7">86.49%</td>
</tr>
<tr class="row-108 even">
	<td class="column-1">107</td><td class="column-2">Laburnum Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">771</td><td class="column-7">86.47%</td>
</tr>
<tr class="row-109 odd">
	<td class="column-1">108</td><td class="column-2">Our Lady Help of Christians School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">316</td><td class="column-7">86.47%</td>
</tr>
<tr class="row-110 even">
	<td class="column-1">109</td><td class="column-2">Heidelberg Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">517</td><td class="column-7">86.46%</td>
</tr>
<tr class="row-111 odd">
	<td class="column-1">110</td><td class="column-2">Heathdale Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1237</td><td class="column-7">86.46%</td>
</tr>
<tr class="row-112 even">
	<td class="column-1">111</td><td class="column-2">Mount Waverley Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">801</td><td class="column-7">86.44%</td>
</tr>
<tr class="row-113 odd">
	<td class="column-1">112</td><td class="column-2">Ormond Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">414</td><td class="column-7">86.37%</td>
</tr>
<tr class="row-114 even">
	<td class="column-1">113</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">341</td><td class="column-7">86.37%</td>
</tr>
<tr class="row-115 odd">
	<td class="column-1">114</td><td class="column-2">Heatherton Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">370</td><td class="column-7">86.37%</td>
</tr>
<tr class="row-116 even">
	<td class="column-1">115</td><td class="column-2">Westgarth Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">647</td><td class="column-7">86.35%</td>
</tr>
<tr class="row-117 odd">
	<td class="column-1">116</td><td class="column-2">Kensington Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">480</td><td class="column-7">86.34%</td>
</tr>
<tr class="row-118 even">
	<td class="column-1">117</td><td class="column-2">St Catherine's School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">689</td><td class="column-7">86.34%</td>
</tr>
<tr class="row-119 odd">
	<td class="column-1">118</td><td class="column-2">Shelford Girls' Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">504</td><td class="column-7">86.34%</td>
</tr>
<tr class="row-120 even">
	<td class="column-1">119</td><td class="column-2">Panton Hill Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">118</td><td class="column-7">86.25%</td>
</tr>
<tr class="row-121 odd">
	<td class="column-1">120</td><td class="column-2">Glen Iris Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">423</td><td class="column-7">86.22%</td>
</tr>
<tr class="row-122 even">
	<td class="column-1">121</td><td class="column-2">Mount Macedon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">128</td><td class="column-7">86.11%</td>
</tr>
<tr class="row-123 odd">
	<td class="column-1">122</td><td class="column-2">Malvern Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">697</td><td class="column-7">86.11%</td>
</tr>
<tr class="row-124 even">
	<td class="column-1">123</td><td class="column-2">Alphington Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">525</td><td class="column-7">86.08%</td>
</tr>
<tr class="row-125 odd">
	<td class="column-1">124</td><td class="column-2">Auburn South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">674</td><td class="column-7">86.06%</td>
</tr>
<tr class="row-126 even">
	<td class="column-1">125</td><td class="column-2">Hawthorn West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">486</td><td class="column-7">85.96%</td>
</tr>
<tr class="row-127 odd">
	<td class="column-1">126</td><td class="column-2">Penleigh &amp; Essendon Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2446</td><td class="column-7">85.94%</td>
</tr>
<tr class="row-128 even">
	<td class="column-1">127</td><td class="column-2">Kew East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">470</td><td class="column-7">85.93%</td>
</tr>
<tr class="row-129 odd">
	<td class="column-1">128</td><td class="column-2">Parkdale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">665</td><td class="column-7">85.93%</td>
</tr>
<tr class="row-130 even">
	<td class="column-1">129</td><td class="column-2">Blackburn Lake Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">469</td><td class="column-7">85.91%</td>
</tr>
<tr class="row-131 odd">
	<td class="column-1">130</td><td class="column-2">St Kevin's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">127</td><td class="column-7">85.89%</td>
</tr>
<tr class="row-132 even">
	<td class="column-1">131</td><td class="column-2">St Roch's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">169</td><td class="column-7">85.89%</td>
</tr>
<tr class="row-133 odd">
	<td class="column-1">132</td><td class="column-2">Dunkeld Consolidated School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">104</td><td class="column-7">85.84%</td>
</tr>
<tr class="row-134 even">
	<td class="column-1">133</td><td class="column-2">Essendon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">509</td><td class="column-7">85.81%</td>
</tr>
<tr class="row-135 odd">
	<td class="column-1">134</td><td class="column-2">Valkstone Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">676</td><td class="column-7">85.81%</td>
</tr>
<tr class="row-136 even">
	<td class="column-1">135</td><td class="column-2">Mont Albert Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">718</td><td class="column-7">85.77%</td>
</tr>
<tr class="row-137 odd">
	<td class="column-1">136</td><td class="column-2">Yackandandah Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">143</td><td class="column-7">85.75%</td>
</tr>
<tr class="row-138 even">
	<td class="column-1">137</td><td class="column-2">Orchard Grove Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">662</td><td class="column-7">85.75%</td>
</tr>
<tr class="row-139 odd">
	<td class="column-1">138</td><td class="column-2">Gardenvale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">663</td><td class="column-7">85.72%</td>
</tr>
<tr class="row-140 even">
	<td class="column-1">139</td><td class="column-2">Malvern Valley Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">253</td><td class="column-7">85.72%</td>
</tr>
<tr class="row-141 odd">
	<td class="column-1">140</td><td class="column-2">The King David School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">595</td><td class="column-7">85.68%</td>
</tr>
<tr class="row-142 even">
	<td class="column-1">141</td><td class="column-2">Surrey Hills Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">404</td><td class="column-7">85.65%</td>
</tr>
<tr class="row-143 odd">
	<td class="column-1">142</td><td class="column-2">Balwyn North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">479</td><td class="column-7">85.65%</td>
</tr>
<tr class="row-144 even">
	<td class="column-1">143</td><td class="column-2">Doreen Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">151</td><td class="column-7">85.58%</td>
</tr>
<tr class="row-145 odd">
	<td class="column-1">144</td><td class="column-2">Clifton Hill Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">746</td><td class="column-7">85.58%</td>
</tr>
<tr class="row-146 even">
	<td class="column-1">145</td><td class="column-2">Strathmore North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">425</td><td class="column-7">85.55%</td>
</tr>
<tr class="row-147 odd">
	<td class="column-1">146</td><td class="column-2">Donvale Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1255</td><td class="column-7">85.51%</td>
</tr>
<tr class="row-148 even">
	<td class="column-1">147</td><td class="column-2">Aireys Inlet Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">109</td><td class="column-7">85.51%</td>
</tr>
<tr class="row-149 odd">
	<td class="column-1">148</td><td class="column-2">Bialik College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">874</td><td class="column-7">85.48%</td>
</tr>
<tr class="row-150 even">
	<td class="column-1">149</td><td class="column-2">Kororoit Creek Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1178</td><td class="column-7">85.48%</td>
</tr>
<tr class="row-151 odd">
	<td class="column-1">150</td><td class="column-2">Ivanhoe Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1913</td><td class="column-7">85.39%</td>
</tr>
<tr class="row-152 even">
	<td class="column-1">151</td><td class="column-2">Deutsche Schule Melbourne</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">103</td><td class="column-7">85.38%</td>
</tr>
<tr class="row-153 odd">
	<td class="column-1">152</td><td class="column-2">Mount Waverley North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">233</td><td class="column-7">85.38%</td>
</tr>
<tr class="row-154 even">
	<td class="column-1">153</td><td class="column-2">Rangeview Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">582</td><td class="column-7">85.36%</td>
</tr>
<tr class="row-155 odd">
	<td class="column-1">154</td><td class="column-2">Mount Pleasant Road Nunawading Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">328</td><td class="column-7">85.34%</td>
</tr>
<tr class="row-156 even">
	<td class="column-1">155</td><td class="column-2">St Michael's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">305</td><td class="column-7">85.34%</td>
</tr>
<tr class="row-157 odd">
	<td class="column-1">156</td><td class="column-2">Inverleigh Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">202</td><td class="column-7">85.32%</td>
</tr>
<tr class="row-158 even">
	<td class="column-1">157</td><td class="column-2">Woorinen District Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">86</td><td class="column-7">85.32%</td>
</tr>
<tr class="row-159 odd">
	<td class="column-1">158</td><td class="column-2">Lloyd Street Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">533</td><td class="column-7">85.31%</td>
</tr>
<tr class="row-160 even">
	<td class="column-1">159</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">344</td><td class="column-7">85.31%</td>
</tr>
<tr class="row-161 odd">
	<td class="column-1">160</td><td class="column-2">Nunawading Christian College - Primary</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">186</td><td class="column-7">85.29%</td>
</tr>
<tr class="row-162 even">
	<td class="column-1">161</td><td class="column-2">Fyans Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">311</td><td class="column-7">85.27%</td>
</tr>
<tr class="row-163 odd">
	<td class="column-1">162</td><td class="column-2">Donvale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">256</td><td class="column-7">85.26%</td>
</tr>
<tr class="row-164 even">
	<td class="column-1">163</td><td class="column-2">Syndal South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">423</td><td class="column-7">85.24%</td>
</tr>
<tr class="row-165 odd">
	<td class="column-1">164</td><td class="column-2">Mountain Gate Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">359</td><td class="column-7">85.22%</td>
</tr>
<tr class="row-166 even">
	<td class="column-1">165</td><td class="column-2">Templestowe Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">455</td><td class="column-7">85.20%</td>
</tr>
<tr class="row-167 odd">
	<td class="column-1">166</td><td class="column-2">Templestowe Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">345</td><td class="column-7">85.19%</td>
</tr>
<tr class="row-168 even">
	<td class="column-1">167</td><td class="column-2">Alamanda K-9 College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">2065</td><td class="column-7">85.19%</td>
</tr>
<tr class="row-169 odd">
	<td class="column-1">168</td><td class="column-2">Brentwood Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">710</td><td class="column-7">85.17%</td>
</tr>
<tr class="row-170 even">
	<td class="column-1">169</td><td class="column-2">St Kilda Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">599</td><td class="column-7">85.15%</td>
</tr>
<tr class="row-171 odd">
	<td class="column-1">170</td><td class="column-2">Brighton Beach Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">379</td><td class="column-7">85.15%</td>
</tr>
<tr class="row-172 even">
	<td class="column-1">171</td><td class="column-2">Southern Cross Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">532</td><td class="column-7">85.15%</td>
</tr>
<tr class="row-173 odd">
	<td class="column-1">172</td><td class="column-2">Cheltenham East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">438</td><td class="column-7">85.14%</td>
</tr>
<tr class="row-174 even">
	<td class="column-1">173</td><td class="column-2">Tintern Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">804</td><td class="column-7">85.14%</td>
</tr>
<tr class="row-175 odd">
	<td class="column-1">174</td><td class="column-2">St Gregory the Great School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">426</td><td class="column-7">85.12%</td>
</tr>
<tr class="row-176 even">
	<td class="column-1">175</td><td class="column-2">Wales Street Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">603</td><td class="column-7">85.08%</td>
</tr>
<tr class="row-177 odd">
	<td class="column-1">176</td><td class="column-2">Sacre Coeur</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">720</td><td class="column-7">85.08%</td>
</tr>
<tr class="row-178 even">
	<td class="column-1">177</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">323</td><td class="column-7">85.08%</td>
</tr>
<tr class="row-179 odd">
	<td class="column-1">178</td><td class="column-2">Our Lady of Perpetual Succour School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">186</td><td class="column-7">85.08%</td>
</tr>
<tr class="row-180 even">
	<td class="column-1">179</td><td class="column-2">Good News Lutheran College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">879</td><td class="column-7">85.08%</td>
</tr>
<tr class="row-181 odd">
	<td class="column-1">180</td><td class="column-2">Mitcham Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">410</td><td class="column-7">85.07%</td>
</tr>
<tr class="row-182 even">
	<td class="column-1">181</td><td class="column-2">Yarra Valley Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1214</td><td class="column-7">85.02%</td>
</tr>
<tr class="row-183 odd">
	<td class="column-1">182</td><td class="column-2">North Melbourne Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">743</td><td class="column-7">85.00%</td>
</tr>
<tr class="row-184 even">
	<td class="column-1">183</td><td class="column-2">Antonio Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">503</td><td class="column-7">85.00%</td>
</tr>
<tr class="row-185 odd">
	<td class="column-1">184</td><td class="column-2">Belmont Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">302</td><td class="column-7">84.98%</td>
</tr>
<tr class="row-186 even">
	<td class="column-1">185</td><td class="column-2">Williamstown North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">766</td><td class="column-7">84.98%</td>
</tr>
<tr class="row-187 odd">
	<td class="column-1">186</td><td class="column-2">Cardross Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">104</td><td class="column-7">84.98%</td>
</tr>
<tr class="row-188 even">
	<td class="column-1">187</td><td class="column-2">Donburn Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">455</td><td class="column-7">84.98%</td>
</tr>
<tr class="row-189 odd">
	<td class="column-1">188</td><td class="column-2">Aspendale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">532</td><td class="column-7">84.93%</td>
</tr>
<tr class="row-190 even">
	<td class="column-1">189</td><td class="column-2">St Vincent de Paul's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">432</td><td class="column-7">84.91%</td>
</tr>
<tr class="row-191 odd">
	<td class="column-1">190</td><td class="column-2">Geelong Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1448</td><td class="column-7">84.90%</td>
</tr>
<tr class="row-192 even">
	<td class="column-1">191</td><td class="column-2">Merri Creek Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">440</td><td class="column-7">84.88%</td>
</tr>
<tr class="row-193 odd">
	<td class="column-1">192</td><td class="column-2">Boroondara Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">632</td><td class="column-7">84.86%</td>
</tr>
<tr class="row-194 even">
	<td class="column-1">193</td><td class="column-2">Fairfield Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">487</td><td class="column-7">84.83%</td>
</tr>
<tr class="row-195 odd">
	<td class="column-1">194</td><td class="column-2">St Leonard's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">277</td><td class="column-7">84.78%</td>
</tr>
<tr class="row-196 even">
	<td class="column-1">195</td><td class="column-2">Blackburn Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">518</td><td class="column-7">84.76%</td>
</tr>
<tr class="row-197 odd">
	<td class="column-1">196</td><td class="column-2">St Clement of Rome School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">354</td><td class="column-7">84.72%</td>
</tr>
<tr class="row-198 even">
	<td class="column-1">197</td><td class="column-2">Beth Rivkah Ladies College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">590</td><td class="column-7">84.72%</td>
</tr>
<tr class="row-199 odd">
	<td class="column-1">198</td><td class="column-2">Overnewton Anglican Community College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2109</td><td class="column-7">84.72%</td>
</tr>
<tr class="row-200 even">
	<td class="column-1">199</td><td class="column-2">Carlton Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">359</td><td class="column-7">84.71%</td>
</tr>
<tr class="row-201 odd">
	<td class="column-1">200</td><td class="column-2">Puckapunyal Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">207</td><td class="column-7">84.69%</td>
</tr>
<tr class="row-202 even">
	<td class="column-1">201</td><td class="column-2">Carrum Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">314</td><td class="column-7">84.69%</td>
</tr>
<tr class="row-203 odd">
	<td class="column-1">202</td><td class="column-2">Wesley College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">3031</td><td class="column-7">84.69%</td>
</tr>
<tr class="row-204 even">
	<td class="column-1">203</td><td class="column-2">Sacred Heart School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">267</td><td class="column-7">84.67%</td>
</tr>
<tr class="row-205 odd">
	<td class="column-1">204</td><td class="column-2">St Leo the Great School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">242</td><td class="column-7">84.67%</td>
</tr>
<tr class="row-206 even">
	<td class="column-1">205</td><td class="column-2">Bacchus Marsh Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">1986</td><td class="column-7">84.67%</td>
</tr>
<tr class="row-207 odd">
	<td class="column-1">206</td><td class="column-2">Oakleigh Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">436</td><td class="column-7">84.65%</td>
</tr>
<tr class="row-208 even">
	<td class="column-1">207</td><td class="column-2">Lowther Hall Anglican Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">785</td><td class="column-7">84.65%</td>
</tr>
<tr class="row-209 odd">
	<td class="column-1">208</td><td class="column-2">Galilee Regional Catholic Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">313</td><td class="column-7">84.60%</td>
</tr>
<tr class="row-210 even">
	<td class="column-1">209</td><td class="column-2">Milgate Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">646</td><td class="column-7">84.59%</td>
</tr>
<tr class="row-211 odd">
	<td class="column-1">210</td><td class="column-2">Derinya Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">800</td><td class="column-7">84.57%</td>
</tr>
<tr class="row-212 even">
	<td class="column-1">211</td><td class="column-2">St Paul's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">271</td><td class="column-7">84.55%</td>
</tr>
<tr class="row-213 odd">
	<td class="column-1">212</td><td class="column-2">The Kilmore International School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">437</td><td class="column-7">84.53%</td>
</tr>
<tr class="row-214 even">
	<td class="column-1">213</td><td class="column-2">Altona Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">631</td><td class="column-7">84.52%</td>
</tr>
<tr class="row-215 odd">
	<td class="column-1">214</td><td class="column-2">Doncaster Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">548</td><td class="column-7">84.50%</td>
</tr>
<tr class="row-216 even">
	<td class="column-1">215</td><td class="column-2">Mount Eliza North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">626</td><td class="column-7">84.50%</td>
</tr>
<tr class="row-217 odd">
	<td class="column-1">216</td><td class="column-2">St Margaret's Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">333</td><td class="column-7">84.43%</td>
</tr>
<tr class="row-218 even">
	<td class="column-1">217</td><td class="column-2">Oakleigh Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">674</td><td class="column-7">84.40%</td>
</tr>
<tr class="row-219 odd">
	<td class="column-1">218</td><td class="column-2">Carlton North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">285</td><td class="column-7">84.38%</td>
</tr>
<tr class="row-220 even">
	<td class="column-1">219</td><td class="column-2">Our Holy Redeemer School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">163</td><td class="column-7">84.38%</td>
</tr>
<tr class="row-221 odd">
	<td class="column-1">220</td><td class="column-2">Oxley Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">827</td><td class="column-7">84.38%</td>
</tr>
<tr class="row-222 even">
	<td class="column-1">221</td><td class="column-2">Hughesdale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">542</td><td class="column-7">84.36%</td>
</tr>
<tr class="row-223 odd">
	<td class="column-1">222</td><td class="column-2">Wembley Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">720</td><td class="column-7">84.35%</td>
</tr>
<tr class="row-224 even">
	<td class="column-1">223</td><td class="column-2">Tambo Upper Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">103</td><td class="column-7">84.33%</td>
</tr>
<tr class="row-225 odd">
	<td class="column-1">224</td><td class="column-2">Ocean Grove Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">633</td><td class="column-7">84.33%</td>
</tr>
<tr class="row-226 even">
	<td class="column-1">225</td><td class="column-2">Moonee Ponds Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">351</td><td class="column-7">84.33%</td>
</tr>
<tr class="row-227 odd">
	<td class="column-1">226</td><td class="column-2">Caulfield Junior College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">474</td><td class="column-7">84.31%</td>
</tr>
<tr class="row-228 even">
	<td class="column-1">227</td><td class="column-2">Aberfeldie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">378</td><td class="column-7">84.26%</td>
</tr>
<tr class="row-229 odd">
	<td class="column-1">228</td><td class="column-2">Newport Lakes Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">445</td><td class="column-7">84.24%</td>
</tr>
<tr class="row-230 even">
	<td class="column-1">229</td><td class="column-2">Haddon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">222</td><td class="column-7">84.24%</td>
</tr>
<tr class="row-231 odd">
	<td class="column-1">230</td><td class="column-2">St Bernadette's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">239</td><td class="column-7">84.24%</td>
</tr>
<tr class="row-232 even">
	<td class="column-1">231</td><td class="column-2">St Paul's Anglican Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">1259</td><td class="column-7">84.24%</td>
</tr>
<tr class="row-233 odd">
	<td class="column-1">232</td><td class="column-2">Cornish College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">656</td><td class="column-7">84.24%</td>
</tr>
<tr class="row-234 even">
	<td class="column-1">233</td><td class="column-2">Coburg West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">504</td><td class="column-7">84.23%</td>
</tr>
<tr class="row-235 odd">
	<td class="column-1">234</td><td class="column-2">Tulliallan Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">337</td><td class="column-7">84.21%</td>
</tr>
<tr class="row-236 even">
	<td class="column-1">235</td><td class="column-2">Footscray North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">409</td><td class="column-7">84.17%</td>
</tr>
<tr class="row-237 odd">
	<td class="column-1">236</td><td class="column-2">Heathmont East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">584</td><td class="column-7">84.16%</td>
</tr>
<tr class="row-238 even">
	<td class="column-1">237</td><td class="column-2">Moorabbin Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">248</td><td class="column-7">84.14%</td>
</tr>
<tr class="row-239 odd">
	<td class="column-1">238</td><td class="column-2">Viewbank Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">654</td><td class="column-7">84.14%</td>
</tr>
<tr class="row-240 even">
	<td class="column-1">239</td><td class="column-2">Essex Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">567</td><td class="column-7">84.14%</td>
</tr>
<tr class="row-241 odd">
	<td class="column-1">240</td><td class="column-2">Wonga Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">397</td><td class="column-7">84.12%</td>
</tr>
<tr class="row-242 even">
	<td class="column-1">241</td><td class="column-2">Andersons Creek Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">339</td><td class="column-7">84.12%</td>
</tr>
<tr class="row-243 odd">
	<td class="column-1">242</td><td class="column-2">Middle Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">531</td><td class="column-7">84.11%</td>
</tr>
<tr class="row-244 even">
	<td class="column-1">243</td><td class="column-2">Emerald Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">502</td><td class="column-7">84.11%</td>
</tr>
<tr class="row-245 odd">
	<td class="column-1">244</td><td class="column-2">Montmorency South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">619</td><td class="column-7">84.11%</td>
</tr>
<tr class="row-246 even">
	<td class="column-1">245</td><td class="column-2">Apollo Bay P-12 College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">262</td><td class="column-7">84.11%</td>
</tr>
<tr class="row-247 odd">
	<td class="column-1">246</td><td class="column-2">Coleraine Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">48</td><td class="column-7">84.09%</td>
</tr>
<tr class="row-248 even">
	<td class="column-1">247</td><td class="column-2">Ripponlea Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">289</td><td class="column-7">84.09%</td>
</tr>
<tr class="row-249 odd">
	<td class="column-1">248</td><td class="column-2">St James' School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">337</td><td class="column-7">84.09%</td>
</tr>
<tr class="row-250 even">
	<td class="column-1">249</td><td class="column-2">Ascot Vale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">266</td><td class="column-7">84.07%</td>
</tr>
<tr class="row-251 odd">
	<td class="column-1">250</td><td class="column-2">Preston Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">741</td><td class="column-7">84.04%</td>
</tr>
<tr class="row-252 even">
	<td class="column-1">251</td><td class="column-2">Bellaire Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">607</td><td class="column-7">84.04%</td>
</tr>
<tr class="row-253 odd">
	<td class="column-1">252</td><td class="column-2">St Robert's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">381</td><td class="column-7">84.04%</td>
</tr>
<tr class="row-254 even">
	<td class="column-1">253</td><td class="column-2">Croydon Hills Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">680</td><td class="column-7">84.02%</td>
</tr>
<tr class="row-255 odd">
	<td class="column-1">254</td><td class="column-2">St Luke the Evangelist School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">202</td><td class="column-7">84.02%</td>
</tr>
<tr class="row-256 even">
	<td class="column-1">255</td><td class="column-2">Beechworth Montessori School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">81</td><td class="column-7">84.02%</td>
</tr>
<tr class="row-257 odd">
	<td class="column-1">256</td><td class="column-2">St Bridget's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">62</td><td class="column-7">84.00%</td>
</tr>
<tr class="row-258 even">
	<td class="column-1">257</td><td class="column-2">Warrandyte Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">211</td><td class="column-7">83.99%</td>
</tr>
<tr class="row-259 odd">
	<td class="column-1">258</td><td class="column-2">Cheltenham Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">419</td><td class="column-7">83.99%</td>
</tr>
<tr class="row-260 even">
	<td class="column-1">259</td><td class="column-2">Greenhills Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">526</td><td class="column-7">83.99%</td>
</tr>
<tr class="row-261 odd">
	<td class="column-1">260</td><td class="column-2">Richmond Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">351</td><td class="column-7">83.99%</td>
</tr>
<tr class="row-262 even">
	<td class="column-1">261</td><td class="column-2">Essendon North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">664</td><td class="column-7">83.97%</td>
</tr>
<tr class="row-263 odd">
	<td class="column-1">262</td><td class="column-2">Jells Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">507</td><td class="column-7">83.95%</td>
</tr>
<tr class="row-264 even">
	<td class="column-1">263</td><td class="column-2">Mentone Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1435</td><td class="column-7">83.93%</td>
</tr>
<tr class="row-265 odd">
	<td class="column-1">264</td><td class="column-2">Northside Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">319</td><td class="column-7">83.93%</td>
</tr>
<tr class="row-266 even">
	<td class="column-1">265</td><td class="column-2">Strathmore Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">505</td><td class="column-7">83.92%</td>
</tr>
<tr class="row-267 odd">
	<td class="column-1">266</td><td class="column-2">Marysville Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">56</td><td class="column-7">83.90%</td>
</tr>
<tr class="row-268 even">
	<td class="column-1">267</td><td class="column-2">Mount Blowhard Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">71</td><td class="column-7">83.90%</td>
</tr>
<tr class="row-269 odd">
	<td class="column-1">268</td><td class="column-2">Beaumaris Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">543</td><td class="column-7">83.88%</td>
</tr>
<tr class="row-270 even">
	<td class="column-1">269</td><td class="column-2">St Philip's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">125</td><td class="column-7">83.88%</td>
</tr>
<tr class="row-271 odd">
	<td class="column-1">270</td><td class="column-2">Coburg North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">415</td><td class="column-7">83.85%</td>
</tr>
<tr class="row-272 even">
	<td class="column-1">271</td><td class="column-2">Coatesville Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">759</td><td class="column-7">83.83%</td>
</tr>
<tr class="row-273 odd">
	<td class="column-1">272</td><td class="column-2">Peninsula Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1282</td><td class="column-7">83.83%</td>
</tr>
<tr class="row-274 even">
	<td class="column-1">273</td><td class="column-2">Maranatha Christian School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">766</td><td class="column-7">83.83%</td>
</tr>
<tr class="row-275 odd">
	<td class="column-1">274</td><td class="column-2">St Christopher's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">91</td><td class="column-7">83.81%</td>
</tr>
<tr class="row-276 even">
	<td class="column-1">275</td><td class="column-2">St Michael's Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1203</td><td class="column-7">83.81%</td>
</tr>
<tr class="row-277 odd">
	<td class="column-1">276</td><td class="column-2">St Louis de Montfort's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">725</td><td class="column-7">83.80%</td>
</tr>
<tr class="row-278 even">
	<td class="column-1">277</td><td class="column-2">Yesodei HaTorah College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">163</td><td class="column-7">83.80%</td>
</tr>
<tr class="row-279 odd">
	<td class="column-1">278</td><td class="column-2">Solway Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">507</td><td class="column-7">83.78%</td>
</tr>
<tr class="row-280 even">
	<td class="column-1">279</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Outer Regional</td><td class="column-6">516</td><td class="column-7">83.76%</td>
</tr>
<tr class="row-281 odd">
	<td class="column-1">280</td><td class="column-2">St Thomas the Apostle School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">350</td><td class="column-7">83.76%</td>
</tr>
<tr class="row-282 even">
	<td class="column-1">281</td><td class="column-2">Lighthouse Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">677</td><td class="column-7">83.76%</td>
</tr>
<tr class="row-283 odd">
	<td class="column-1">282</td><td class="column-2">Mckinnon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">814</td><td class="column-7">83.75%</td>
</tr>
<tr class="row-284 even">
	<td class="column-1">283</td><td class="column-2">Watsonia Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">308</td><td class="column-7">83.75%</td>
</tr>
<tr class="row-285 odd">
	<td class="column-1">284</td><td class="column-2">Xavier College</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">1961</td><td class="column-7">83.75%</td>
</tr>
<tr class="row-286 even">
	<td class="column-1">285</td><td class="column-2">Northcote Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">377</td><td class="column-7">83.71%</td>
</tr>
<tr class="row-287 odd">
	<td class="column-1">286</td><td class="column-2">Apollo Parkways Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">819</td><td class="column-7">83.71%</td>
</tr>
<tr class="row-288 even">
	<td class="column-1">287</td><td class="column-2">Sacred Heart School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">276</td><td class="column-7">83.66%</td>
</tr>
<tr class="row-289 odd">
	<td class="column-1">288</td><td class="column-2">Mount Scopus Memorial College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1283</td><td class="column-7">83.66%</td>
</tr>
<tr class="row-290 even">
	<td class="column-1">289</td><td class="column-2">Fish Creek and District Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">89</td><td class="column-7">83.63%</td>
</tr>
<tr class="row-291 odd">
	<td class="column-1">290</td><td class="column-2">Glen Huntly Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">315</td><td class="column-7">83.63%</td>
</tr>
<tr class="row-292 even">
	<td class="column-1">291</td><td class="column-2">Edinburgh College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">276</td><td class="column-7">83.63%</td>
</tr>
<tr class="row-293 odd">
	<td class="column-1">292</td><td class="column-2">Thomas Mitchell Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">803</td><td class="column-7">83.61%</td>
</tr>
<tr class="row-294 even">
	<td class="column-1">293</td><td class="column-2">St Anne's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">289</td><td class="column-7">83.61%</td>
</tr>
<tr class="row-295 odd">
	<td class="column-1">294</td><td class="column-2">Eltham North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">593</td><td class="column-7">83.57%</td>
</tr>
<tr class="row-296 even">
	<td class="column-1">295</td><td class="column-2">Sandringham East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">522</td><td class="column-7">83.56%</td>
</tr>
<tr class="row-297 odd">
	<td class="column-1">296</td><td class="column-2">Woodleigh School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">945</td><td class="column-7">83.56%</td>
</tr>
<tr class="row-298 even">
	<td class="column-1">297</td><td class="column-2">Gordon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">112</td><td class="column-7">83.54%</td>
</tr>
<tr class="row-299 odd">
	<td class="column-1">298</td><td class="column-2">Seabrook Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">980</td><td class="column-7">83.54%</td>
</tr>
<tr class="row-300 even">
	<td class="column-1">299</td><td class="column-2">Belle Vue Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">174</td><td class="column-7">83.52%</td>
</tr>
<tr class="row-301 odd">
	<td class="column-1">300</td><td class="column-2">St Mark's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">478</td><td class="column-7">83.52%</td>
</tr>
<tr class="row-302 even">
	<td class="column-1">301</td><td class="column-2">Mulgrave Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">354</td><td class="column-7">83.50%</td>
</tr>
<tr class="row-303 odd">
	<td class="column-1">302</td><td class="column-2">St James' School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">249</td><td class="column-7">83.50%</td>
</tr>
<tr class="row-304 even">
	<td class="column-1">303</td><td class="column-2">Magpie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">97</td><td class="column-7">83.47%</td>
</tr>
<tr class="row-305 odd">
	<td class="column-1">304</td><td class="column-2">Kingston Heath Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">296</td><td class="column-7">83.45%</td>
</tr>
<tr class="row-306 even">
	<td class="column-1">305</td><td class="column-2">St Aloysius' School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">61</td><td class="column-7">83.45%</td>
</tr>
<tr class="row-307 odd">
	<td class="column-1">306</td><td class="column-2">Albert Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">581</td><td class="column-7">83.44%</td>
</tr>
<tr class="row-308 even">
	<td class="column-1">307</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">410</td><td class="column-7">83.42%</td>
</tr>
<tr class="row-309 odd">
	<td class="column-1">308</td><td class="column-2">Mordialloc Beach Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">183</td><td class="column-7">83.40%</td>
</tr>
<tr class="row-310 even">
	<td class="column-1">309</td><td class="column-2">Elwood Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">803</td><td class="column-7">83.40%</td>
</tr>
<tr class="row-311 odd">
	<td class="column-1">310</td><td class="column-2">Chatham Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">409</td><td class="column-7">83.40%</td>
</tr>
<tr class="row-312 even">
	<td class="column-1">311</td><td class="column-2">Ivanhoe East Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">546</td><td class="column-7">83.37%</td>
</tr>
<tr class="row-313 odd">
	<td class="column-1">312</td><td class="column-2">Kinglake Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">86</td><td class="column-7">83.35%</td>
</tr>
<tr class="row-314 even">
	<td class="column-1">313</td><td class="column-2">Templestowe Valley Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">488</td><td class="column-7">83.35%</td>
</tr>
<tr class="row-315 odd">
	<td class="column-1">314</td><td class="column-2">Whitehorse Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">361</td><td class="column-7">83.33%</td>
</tr>
<tr class="row-316 even">
	<td class="column-1">315</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Outer Regional</td><td class="column-6">42</td><td class="column-7">83.32%</td>
</tr>
<tr class="row-317 odd">
	<td class="column-1">316</td><td class="column-2">Brunswick South West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">404</td><td class="column-7">83.30%</td>
</tr>
<tr class="row-318 even">
	<td class="column-1">317</td><td class="column-2">Wantirna South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">265</td><td class="column-7">83.30%</td>
</tr>
<tr class="row-319 odd">
	<td class="column-1">318</td><td class="column-2">Ringwood Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">271</td><td class="column-7">83.30%</td>
</tr>
<tr class="row-320 even">
	<td class="column-1">319</td><td class="column-2">Selby Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">240</td><td class="column-7">83.28%</td>
</tr>
<tr class="row-321 odd">
	<td class="column-1">320</td><td class="column-2">Bayswater South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">303</td><td class="column-7">83.26%</td>
</tr>
<tr class="row-322 even">
	<td class="column-1">321</td><td class="column-2">Westbreen Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">263</td><td class="column-7">83.25%</td>
</tr>
<tr class="row-323 odd">
	<td class="column-1">322</td><td class="column-2">Tinternvale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">334</td><td class="column-7">83.25%</td>
</tr>
<tr class="row-324 even">
	<td class="column-1">323</td><td class="column-2">Pleasant Street Primary School ( Ballarat)</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">346</td><td class="column-7">83.23%</td>
</tr>
<tr class="row-325 odd">
	<td class="column-1">324</td><td class="column-2">Wandin Yallock Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">85</td><td class="column-7">83.23%</td>
</tr>
<tr class="row-326 even">
	<td class="column-1">325</td><td class="column-2">Port Melbourne Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">813</td><td class="column-7">83.23%</td>
</tr>
<tr class="row-327 odd">
	<td class="column-1">326</td><td class="column-2">Avondale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">361</td><td class="column-7">83.23%</td>
</tr>
<tr class="row-328 even">
	<td class="column-1">327</td><td class="column-2">Camberwell South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">481</td><td class="column-7">83.21%</td>
</tr>
<tr class="row-329 odd">
	<td class="column-1">328</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">261</td><td class="column-7">83.21%</td>
</tr>
<tr class="row-330 even">
	<td class="column-1">329</td><td class="column-2">Resurrection House</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">159</td><td class="column-7">83.20%</td>
</tr>
<tr class="row-331 odd">
	<td class="column-1">330</td><td class="column-2">Aspendale Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">670</td><td class="column-7">83.18%</td>
</tr>
<tr class="row-332 even">
	<td class="column-1">331</td><td class="column-2">Bright P-12 College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">550</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-333 odd">
	<td class="column-1">332</td><td class="column-2">Macedon Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">241</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-334 even">
	<td class="column-1">333</td><td class="column-2">Castlemaine North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">242</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-335 odd">
	<td class="column-1">334</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Outer Regional</td><td class="column-6">42</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-336 even">
	<td class="column-1">335</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">365</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-337 odd">
	<td class="column-1">336</td><td class="column-2">St Finbar's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">324</td><td class="column-7">83.16%</td>
</tr>
<tr class="row-338 even">
	<td class="column-1">337</td><td class="column-2">St Joseph's Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">156</td><td class="column-7">83.13%</td>
</tr>
<tr class="row-339 odd">
	<td class="column-1">338</td><td class="column-2">Red Hill Consolidated School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">578</td><td class="column-7">83.11%</td>
</tr>
<tr class="row-340 even">
	<td class="column-1">339</td><td class="column-2">Nazareth School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">314</td><td class="column-7">83.09%</td>
</tr>
<tr class="row-341 odd">
	<td class="column-1">340</td><td class="column-2">Hillcrest Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1524</td><td class="column-7">83.09%</td>
</tr>
<tr class="row-342 even">
	<td class="column-1">341</td><td class="column-2">Flemington Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">414</td><td class="column-7">83.08%</td>
</tr>
<tr class="row-343 odd">
	<td class="column-1">342</td><td class="column-2">Brighton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">768</td><td class="column-7">83.08%</td>
</tr>
<tr class="row-344 even">
	<td class="column-1">343</td><td class="column-2">St Columba's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">154</td><td class="column-7">83.08%</td>
</tr>
<tr class="row-345 odd">
	<td class="column-1">344</td><td class="column-2">Auburn Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">499</td><td class="column-7">83.06%</td>
</tr>
<tr class="row-346 even">
	<td class="column-1">345</td><td class="column-2">St Peter's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">666</td><td class="column-7">83.06%</td>
</tr>
<tr class="row-347 odd">
	<td class="column-1">346</td><td class="column-2">St Patrick's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">338</td><td class="column-7">83.06%</td>
</tr>
<tr class="row-348 even">
	<td class="column-1">347</td><td class="column-2">Balnarring Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">377</td><td class="column-7">83.02%</td>
</tr>
<tr class="row-349 odd">
	<td class="column-1">348</td><td class="column-2">Briar Hill Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">193</td><td class="column-7">83.02%</td>
</tr>
<tr class="row-350 even">
	<td class="column-1">349</td><td class="column-2">St Raphael's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">272</td><td class="column-7">83.02%</td>
</tr>
<tr class="row-351 odd">
	<td class="column-1">350</td><td class="column-2">Our Lady of the Pines School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">348</td><td class="column-7">83.02%</td>
</tr>
<tr class="row-352 even">
	<td class="column-1">351</td><td class="column-2">Williamstown Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">531</td><td class="column-7">83.01%</td>
</tr>
<tr class="row-353 odd">
	<td class="column-1">352</td><td class="column-2">Parktone Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">553</td><td class="column-7">83.01%</td>
</tr>
<tr class="row-354 even">
	<td class="column-1">353</td><td class="column-2">Sandringham Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">488</td><td class="column-7">82.99%</td>
</tr>
<tr class="row-355 odd">
	<td class="column-1">354</td><td class="column-2">Toolamba Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">168</td><td class="column-7">82.99%</td>
</tr>
<tr class="row-356 even">
	<td class="column-1">355</td><td class="column-2">Caulfield South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">496</td><td class="column-7">82.99%</td>
</tr>
<tr class="row-357 odd">
	<td class="column-1">356</td><td class="column-2">Kew Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">493</td><td class="column-7">82.97%</td>
</tr>
<tr class="row-358 even">
	<td class="column-1">357</td><td class="column-2">Dandenong North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">833</td><td class="column-7">82.96%</td>
</tr>
<tr class="row-359 odd">
	<td class="column-1">358</td><td class="column-2">St Monica's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">134</td><td class="column-7">82.94%</td>
</tr>
<tr class="row-360 even">
	<td class="column-1">359</td><td class="column-2">Christian College Highton</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">984</td><td class="column-7">82.92%</td>
</tr>
<tr class="row-361 odd">
	<td class="column-1">360</td><td class="column-2">Montmorency Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">258</td><td class="column-7">82.90%</td>
</tr>
<tr class="row-362 even">
	<td class="column-1">361</td><td class="column-2">Park Ridge Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">612</td><td class="column-7">82.90%</td>
</tr>
<tr class="row-363 odd">
	<td class="column-1">362</td><td class="column-2">Woodend Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">400</td><td class="column-7">82.89%</td>
</tr>
<tr class="row-364 even">
	<td class="column-1">363</td><td class="column-2">Kunyung Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">689</td><td class="column-7">82.89%</td>
</tr>
<tr class="row-365 odd">
	<td class="column-1">364</td><td class="column-2">St Patrick's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">357</td><td class="column-7">82.87%</td>
</tr>
<tr class="row-366 even">
	<td class="column-1">365</td><td class="column-2">Kardinia International College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1787</td><td class="column-7">82.87%</td>
</tr>
<tr class="row-367 odd">
	<td class="column-1">366</td><td class="column-2">Cowes Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">585</td><td class="column-7">82.85%</td>
</tr>
<tr class="row-368 even">
	<td class="column-1">367</td><td class="column-2">Rosanna Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">314</td><td class="column-7">82.85%</td>
</tr>
<tr class="row-369 odd">
	<td class="column-1">368</td><td class="column-2">Bonbeach Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">302</td><td class="column-7">82.85%</td>
</tr>
<tr class="row-370 even">
	<td class="column-1">369</td><td class="column-2">Greensborough Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">168</td><td class="column-7">82.84%</td>
</tr>
<tr class="row-371 odd">
	<td class="column-1">370</td><td class="column-2">Warragul North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">617</td><td class="column-7">82.84%</td>
</tr>
<tr class="row-372 even">
	<td class="column-1">371</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">289</td><td class="column-7">82.84%</td>
</tr>
<tr class="row-373 odd">
	<td class="column-1">372</td><td class="column-2">Birregurra Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">83</td><td class="column-7">82.82%</td>
</tr>
<tr class="row-374 even">
	<td class="column-1">373</td><td class="column-2">Our Lady of the Nativity School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">396</td><td class="column-7">82.82%</td>
</tr>
<tr class="row-375 odd">
	<td class="column-1">374</td><td class="column-2">Marlborough Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">208</td><td class="column-7">82.80%</td>
</tr>
<tr class="row-376 even">
	<td class="column-1">375</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">152</td><td class="column-7">82.80%</td>
</tr>
<tr class="row-377 odd">
	<td class="column-1">376</td><td class="column-2">St Francis Xavier's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">354</td><td class="column-7">82.80%</td>
</tr>
<tr class="row-378 even">
	<td class="column-1">377</td><td class="column-2">Wallington Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">160</td><td class="column-7">82.78%</td>
</tr>
<tr class="row-379 odd">
	<td class="column-1">378</td><td class="column-2">Elsternwick Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">595</td><td class="column-7">82.77%</td>
</tr>
<tr class="row-380 even">
	<td class="column-1">379</td><td class="column-2">Girton Grammar School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">1141</td><td class="column-7">82.77%</td>
</tr>
<tr class="row-381 odd">
	<td class="column-1">380</td><td class="column-2">Tucker Road Bentleigh Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">615</td><td class="column-7">82.75%</td>
</tr>
<tr class="row-382 even">
	<td class="column-1">381</td><td class="column-2">St Paul's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">364</td><td class="column-7">82.75%</td>
</tr>
<tr class="row-383 odd">
	<td class="column-1">382</td><td class="column-2">St James School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Inner Regional</td><td class="column-6">102</td><td class="column-7">82.73%</td>
</tr>
<tr class="row-384 even">
	<td class="column-1">383</td><td class="column-2">Truganina P-9 College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">897</td><td class="column-7">82.73%</td>
</tr>
<tr class="row-385 odd">
	<td class="column-1">384</td><td class="column-2">St Martin of Tours School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">512</td><td class="column-7">82.72%</td>
</tr>
<tr class="row-386 even">
	<td class="column-1">385</td><td class="column-2">Bellbrae Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">501</td><td class="column-7">82.68%</td>
</tr>
<tr class="row-387 odd">
	<td class="column-1">386</td><td class="column-2">St Paul's Kealba Catholic School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">340</td><td class="column-7">82.68%</td>
</tr>
<tr class="row-388 even">
	<td class="column-1">387</td><td class="column-2">St Therese's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">546</td><td class="column-7">82.65%</td>
</tr>
<tr class="row-389 odd">
	<td class="column-1">388</td><td class="column-2">Yeshivah College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">386</td><td class="column-7">82.65%</td>
</tr>
<tr class="row-390 even">
	<td class="column-1">389</td><td class="column-2">King's College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">212</td><td class="column-7">82.65%</td>
</tr>
<tr class="row-391 odd">
	<td class="column-1">390</td><td class="column-2">St John's Lutheran School Geelong</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">248</td><td class="column-7">82.63%</td>
</tr>
<tr class="row-392 even">
	<td class="column-1">391</td><td class="column-2">Glen Waverley South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">338</td><td class="column-7">82.61%</td>
</tr>
<tr class="row-393 odd">
	<td class="column-1">392</td><td class="column-2">Beaconhills College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2902</td><td class="column-7">82.61%</td>
</tr>
<tr class="row-394 even">
	<td class="column-1">393</td><td class="column-2">St Theresa's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">226</td><td class="column-7">82.60%</td>
</tr>
<tr class="row-395 odd">
	<td class="column-1">394</td><td class="column-2">St Damian's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">432</td><td class="column-7">82.58%</td>
</tr>
<tr class="row-396 even">
	<td class="column-1">395</td><td class="column-2">Beaumaris North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">693</td><td class="column-7">82.56%</td>
</tr>
<tr class="row-397 odd">
	<td class="column-1">396</td><td class="column-2">Sacred Heart School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Outer Regional</td><td class="column-6">58</td><td class="column-7">82.54%</td>
</tr>
<tr class="row-398 even">
	<td class="column-1">397</td><td class="column-2">Diamond Creek Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">221</td><td class="column-7">82.53%</td>
</tr>
<tr class="row-399 odd">
	<td class="column-1">398</td><td class="column-2">Quarry Hill Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">361</td><td class="column-7">82.53%</td>
</tr>
<tr class="row-400 even">
	<td class="column-1">399</td><td class="column-2">Footscray West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">619</td><td class="column-7">82.53%</td>
</tr>
<tr class="row-401 odd">
	<td class="column-1">400</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">187</td><td class="column-7">82.53%</td>
</tr>
<tr class="row-402 even">
	<td class="column-1">401</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">319</td><td class="column-7">82.51%</td>
</tr>
<tr class="row-403 odd">
	<td class="column-1">402</td><td class="column-2">St Francis of Assisi School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">1112</td><td class="column-7">82.51%</td>
</tr>
<tr class="row-404 even">
	<td class="column-1">403</td><td class="column-2">Flinders Christian Community College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1877</td><td class="column-7">82.51%</td>
</tr>
<tr class="row-405 odd">
	<td class="column-1">404</td><td class="column-2">Hume Anglican Grammar</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1018</td><td class="column-7">82.51%</td>
</tr>
<tr class="row-406 even">
	<td class="column-1">405</td><td class="column-2">Wandin North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">248</td><td class="column-7">82.49%</td>
</tr>
<tr class="row-407 odd">
	<td class="column-1">406</td><td class="column-2">Port Fairy Consolidated School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">221</td><td class="column-7">82.48%</td>
</tr>
<tr class="row-408 even">
	<td class="column-1">407</td><td class="column-2">St Anne's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">161</td><td class="column-7">82.48%</td>
</tr>
<tr class="row-409 odd">
	<td class="column-1">408</td><td class="column-2">Bendigo Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">172</td><td class="column-7">82.44%</td>
</tr>
<tr class="row-410 even">
	<td class="column-1">409</td><td class="column-2">Kingswood Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">587</td><td class="column-7">82.44%</td>
</tr>
<tr class="row-411 odd">
	<td class="column-1">410</td><td class="column-2">Roberts McCubbin Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">418</td><td class="column-7">82.44%</td>
</tr>
<tr class="row-412 even">
	<td class="column-1">411</td><td class="column-2">Newport Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">481</td><td class="column-7">82.42%</td>
</tr>
<tr class="row-413 odd">
	<td class="column-1">412</td><td class="column-2">Tarrington Lutheran School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Outer Regional</td><td class="column-6">118</td><td class="column-7">82.41%</td>
</tr>
<tr class="row-414 even">
	<td class="column-1">413</td><td class="column-2">Bairnsdale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">246</td><td class="column-7">82.35%</td>
</tr>
<tr class="row-415 odd">
	<td class="column-1">414</td><td class="column-2">Pascoe Vale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">521</td><td class="column-7">82.35%</td>
</tr>
<tr class="row-416 even">
	<td class="column-1">415</td><td class="column-2">Barwon Heads Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">492</td><td class="column-7">82.34%</td>
</tr>
<tr class="row-417 odd">
	<td class="column-1">416</td><td class="column-2">St Peter's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">194</td><td class="column-7">82.34%</td>
</tr>
<tr class="row-418 even">
	<td class="column-1">417</td><td class="column-2">Our Lady of the Way School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">144</td><td class="column-7">82.32%</td>
</tr>
<tr class="row-419 odd">
	<td class="column-1">418</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Inner Regional</td><td class="column-6">153</td><td class="column-7">82.32%</td>
</tr>
<tr class="row-420 even">
	<td class="column-1">419</td><td class="column-2">St Christopher's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">717</td><td class="column-7">82.30%</td>
</tr>
<tr class="row-421 odd">
	<td class="column-1">420</td><td class="column-2">Upper Plenty Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">189</td><td class="column-7">82.27%</td>
</tr>
<tr class="row-422 even">
	<td class="column-1">421</td><td class="column-2">Our Lady of Good Counsel School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">254</td><td class="column-7">82.27%</td>
</tr>
<tr class="row-423 odd">
	<td class="column-1">422</td><td class="column-2">Glenroy Central Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">239</td><td class="column-7">82.27%</td>
</tr>
<tr class="row-424 even">
	<td class="column-1">423</td><td class="column-2">Stella Maris School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">320</td><td class="column-7">82.25%</td>
</tr>
<tr class="row-425 odd">
	<td class="column-1">424</td><td class="column-2">St Peter's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">555</td><td class="column-7">82.23%</td>
</tr>
<tr class="row-426 even">
	<td class="column-1">425</td><td class="column-2">Seaholme Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">302</td><td class="column-7">82.18%</td>
</tr>
<tr class="row-427 odd">
	<td class="column-1">426</td><td class="column-2">St Albans Meadows Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">448</td><td class="column-7">82.18%</td>
</tr>
<tr class="row-428 even">
	<td class="column-1">427</td><td class="column-2">Victory Lutheran College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">716</td><td class="column-7">82.18%</td>
</tr>
<tr class="row-429 odd">
	<td class="column-1">428</td><td class="column-2">Lakeside College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">342</td><td class="column-7">82.17%</td>
</tr>
<tr class="row-430 even">
	<td class="column-1">429</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">297</td><td class="column-7">82.15%</td>
</tr>
<tr class="row-431 odd">
	<td class="column-1">430</td><td class="column-2">Chewton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">77</td><td class="column-7">82.13%</td>
</tr>
<tr class="row-432 even">
	<td class="column-1">431</td><td class="column-2">St Thomas More School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Inner Regional</td><td class="column-6">409</td><td class="column-7">82.11%</td>
</tr>
<tr class="row-433 odd">
	<td class="column-1">432</td><td class="column-2">Aitken College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1269</td><td class="column-7">82.11%</td>
</tr>
<tr class="row-434 even">
	<td class="column-1">433</td><td class="column-2">Old Orchard Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">538</td><td class="column-7">82.10%</td>
</tr>
<tr class="row-435 odd">
	<td class="column-1">434</td><td class="column-2">Dorset Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">487</td><td class="column-7">82.10%</td>
</tr>
<tr class="row-436 even">
	<td class="column-1">435</td><td class="column-2">Avenel Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">127</td><td class="column-7">82.06%</td>
</tr>
<tr class="row-437 odd">
	<td class="column-1">436</td><td class="column-2">Cobden Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">178</td><td class="column-7">82.06%</td>
</tr>
<tr class="row-438 even">
	<td class="column-1">437</td><td class="column-2">Holy Rosary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">306</td><td class="column-7">82.05%</td>
</tr>
<tr class="row-439 odd">
	<td class="column-1">438</td><td class="column-2">Darul Ulum College of Victoria</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">1059</td><td class="column-7">82.01%</td>
</tr>
<tr class="row-440 even">
	<td class="column-1">439</td><td class="column-2">Woodford Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">202</td><td class="column-7">81.99%</td>
</tr>
<tr class="row-441 odd">
	<td class="column-1">440</td><td class="column-2">Watsonia North Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">545</td><td class="column-7">81.99%</td>
</tr>
<tr class="row-442 even">
	<td class="column-1">441</td><td class="column-2">Lyndale Greens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">618</td><td class="column-7">81.99%</td>
</tr>
<tr class="row-443 odd">
	<td class="column-1">442</td><td class="column-2">St Bede's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">292</td><td class="column-7">81.99%</td>
</tr>
<tr class="row-444 even">
	<td class="column-1">443</td><td class="column-2">Sirius College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">2472</td><td class="column-7">81.99%</td>
</tr>
<tr class="row-445 odd">
	<td class="column-1">444</td><td class="column-2">Boronia Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">497</td><td class="column-7">81.98%</td>
</tr>
<tr class="row-446 even">
	<td class="column-1">445</td><td class="column-2">Niddrie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">282</td><td class="column-7">81.98%</td>
</tr>
<tr class="row-447 odd">
	<td class="column-1">446</td><td class="column-2">Mary Immaculate School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">121</td><td class="column-7">81.98%</td>
</tr>
<tr class="row-448 even">
	<td class="column-1">447</td><td class="column-2">Kallista Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">163</td><td class="column-7">81.96%</td>
</tr>
<tr class="row-449 odd">
	<td class="column-1">448</td><td class="column-2">Brunswick South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">318</td><td class="column-7">81.91%</td>
</tr>
<tr class="row-450 even">
	<td class="column-1">449</td><td class="column-2">St Paul Apostle South School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">222</td><td class="column-7">81.91%</td>
</tr>
<tr class="row-451 odd">
	<td class="column-1">450</td><td class="column-2">Scoresby Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">219</td><td class="column-7">81.89%</td>
</tr>
<tr class="row-452 even">
	<td class="column-1">451</td><td class="column-2">Kingsville Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">642</td><td class="column-7">81.87%</td>
</tr>
<tr class="row-453 odd">
	<td class="column-1">452</td><td class="column-2">Richmond West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">274</td><td class="column-7">81.87%</td>
</tr>
<tr class="row-454 even">
	<td class="column-1">453</td><td class="column-2">Mount Eliza Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">644</td><td class="column-7">81.86%</td>
</tr>
<tr class="row-455 odd">
	<td class="column-1">454</td><td class="column-2">Huntingdale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">380</td><td class="column-7">81.86%</td>
</tr>
<tr class="row-456 even">
	<td class="column-1">455</td><td class="column-2">Christian College Bellarine</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">441</td><td class="column-7">81.84%</td>
</tr>
<tr class="row-457 odd">
	<td class="column-1">456</td><td class="column-2">Brunswick North West Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">364</td><td class="column-7">81.82%</td>
</tr>
<tr class="row-458 even">
	<td class="column-1">457</td><td class="column-2">St Patrick's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Inner Regional</td><td class="column-6">124</td><td class="column-7">81.81%</td>
</tr>
<tr class="row-459 odd">
	<td class="column-1">458</td><td class="column-2">Upwey Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">89</td><td class="column-7">81.77%</td>
</tr>
<tr class="row-460 even">
	<td class="column-1">459</td><td class="column-2">St John Vianney's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">148</td><td class="column-7">81.77%</td>
</tr>
<tr class="row-461 odd">
	<td class="column-1">460</td><td class="column-2">Wallarano Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">686</td><td class="column-7">81.75%</td>
</tr>
<tr class="row-462 even">
	<td class="column-1">461</td><td class="column-2">Village School</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">72</td><td class="column-7">81.75%</td>
</tr>
<tr class="row-463 odd">
	<td class="column-1">462</td><td class="column-2">Knox Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">288</td><td class="column-7">81.74%</td>
</tr>
<tr class="row-464 even">
	<td class="column-1">463</td><td class="column-2">Holy Name School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">286</td><td class="column-7">81.74%</td>
</tr>
<tr class="row-465 odd">
	<td class="column-1">464</td><td class="column-2">Emmaus Catholic Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">422</td><td class="column-7">81.74%</td>
</tr>
<tr class="row-466 even">
	<td class="column-1">465</td><td class="column-2">Weeden Heights Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">173</td><td class="column-7">81.72%</td>
</tr>
<tr class="row-467 odd">
	<td class="column-1">466</td><td class="column-2">Courtenay Gardens Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">836</td><td class="column-7">81.72%</td>
</tr>
<tr class="row-468 even">
	<td class="column-1">467</td><td class="column-2">St Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">557</td><td class="column-7">81.72%</td>
</tr>
<tr class="row-469 odd">
	<td class="column-1">468</td><td class="column-2">Highton Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">336</td><td class="column-7">81.70%</td>
</tr>
<tr class="row-470 even">
	<td class="column-1">469</td><td class="column-2">Plenty Valley Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">739</td><td class="column-7">81.70%</td>
</tr>
<tr class="row-471 odd">
	<td class="column-1">470</td><td class="column-2">Shepparton Christian College Ltd</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">200</td><td class="column-7">81.70%</td>
</tr>
<tr class="row-472 even">
	<td class="column-1">471</td><td class="column-2">Maiden Gully Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">550</td><td class="column-7">81.69%</td>
</tr>
<tr class="row-473 odd">
	<td class="column-1">472</td><td class="column-2">St Bernadette's Catholic Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">234</td><td class="column-7">81.69%</td>
</tr>
<tr class="row-474 even">
	<td class="column-1">473</td><td class="column-2">Eltham Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">420</td><td class="column-7">81.67%</td>
</tr>
<tr class="row-475 odd">
	<td class="column-1">474</td><td class="column-2">Woori Yallock Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">264</td><td class="column-7">81.67%</td>
</tr>
<tr class="row-476 even">
	<td class="column-1">475</td><td class="column-2">Yarra Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">260</td><td class="column-7">81.67%</td>
</tr>
<tr class="row-477 odd">
	<td class="column-1">476</td><td class="column-2">Ruskin Park Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">351</td><td class="column-7">81.65%</td>
</tr>
<tr class="row-478 even">
	<td class="column-1">477</td><td class="column-2">St Joseph's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Inner Regional</td><td class="column-6">114</td><td class="column-7">81.65%</td>
</tr>
<tr class="row-479 odd">
	<td class="column-1">478</td><td class="column-2">Sacred Heart School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">305</td><td class="column-7">81.65%</td>
</tr>
<tr class="row-480 even">
	<td class="column-1">479</td><td class="column-2">St Bernard's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">225</td><td class="column-7">81.65%</td>
</tr>
<tr class="row-481 odd">
	<td class="column-1">480</td><td class="column-2">Parkmore Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">137</td><td class="column-7">81.62%</td>
</tr>
<tr class="row-482 even">
	<td class="column-1">481</td><td class="column-2">Rolling Hills Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">364</td><td class="column-7">81.62%</td>
</tr>
<tr class="row-483 odd">
	<td class="column-1">482</td><td class="column-2">St Kevin's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">113</td><td class="column-7">81.62%</td>
</tr>
<tr class="row-484 even">
	<td class="column-1">483</td><td class="column-2">St Margaret Mary's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">135</td><td class="column-7">81.60%</td>
</tr>
<tr class="row-485 odd">
	<td class="column-1">484</td><td class="column-2">St Monica's Primary School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">536</td><td class="column-7">81.57%</td>
</tr>
<tr class="row-486 even">
	<td class="column-1">485</td><td class="column-2">St Anthony's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">162</td><td class="column-7">81.57%</td>
</tr>
<tr class="row-487 odd">
	<td class="column-1">486</td><td class="column-2">Tylden Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">115</td><td class="column-7">81.55%</td>
</tr>
<tr class="row-488 even">
	<td class="column-1">487</td><td class="column-2">Irymple South Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Outer Regional</td><td class="column-6">352</td><td class="column-7">81.55%</td>
</tr>
<tr class="row-489 odd">
	<td class="column-1">488</td><td class="column-2">Oatlands Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1089</td><td class="column-7">81.55%</td>
</tr>
<tr class="row-490 even">
	<td class="column-1">489</td><td class="column-2">St John's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">182</td><td class="column-7">81.55%</td>
</tr>
<tr class="row-491 odd">
	<td class="column-1">490</td><td class="column-2">Eltham College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Major Cities</td><td class="column-6">597</td><td class="column-7">81.55%</td>
</tr>
<tr class="row-492 even">
	<td class="column-1">491</td><td class="column-2">Axedale Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">110</td><td class="column-7">81.53%</td>
</tr>
<tr class="row-493 odd">
	<td class="column-1">492</td><td class="column-2">Research Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">182</td><td class="column-7">81.53%</td>
</tr>
<tr class="row-494 even">
	<td class="column-1">493</td><td class="column-2">Katunga Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">89</td><td class="column-7">81.50%</td>
</tr>
<tr class="row-495 odd">
	<td class="column-1">494</td><td class="column-2">Cambridge Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">1031</td><td class="column-7">81.50%</td>
</tr>
<tr class="row-496 even">
	<td class="column-1">495</td><td class="column-2">Keysborough Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">457</td><td class="column-7">81.50%</td>
</tr>
<tr class="row-497 odd">
	<td class="column-1">496</td><td class="column-2">Holy Spirit School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">251</td><td class="column-7">81.50%</td>
</tr>
<tr class="row-498 even">
	<td class="column-1">497</td><td class="column-2">Great Ryrie Primary School</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Major Cities</td><td class="column-6">501</td><td class="column-7">81.48%</td>
</tr>
<tr class="row-499 odd">
	<td class="column-1">498</td><td class="column-2">St Justin's School</td><td class="column-3">VIC</td><td class="column-4">Catholic</td><td class="column-5">Major Cities</td><td class="column-6">347</td><td class="column-7">81.48%</td>
</tr>
<tr class="row-500 even">
	<td class="column-1">499</td><td class="column-2">Wodonga Middle Years College</td><td class="column-3">VIC</td><td class="column-4">Government</td><td class="column-5">Inner Regional</td><td class="column-6">823</td><td class="column-7">81.46%</td>
</tr>
<tr class="row-501 odd">
	<td class="column-1">500</td><td class="column-2">Olivet Christian College</td><td class="column-3">VIC</td><td class="column-4">Independent</td><td class="column-5">Inner Regional</td><td class="column-6">75</td><td class="column-7">81.46%</td>
</tr>
</tbody>
</table>
<!-- #tablepress-233 from cache -->
<p>Share this post:</p>
<p>
<div class="fb-social-plugin fb-like" data-ref="shortcode" data-href="http://www.schoolcatchment.com.au/?p=12301"></div>
<div class="twitter-share"><a href="https://twitter.com/intent/tweet?url=http%3A%2F%2Fwww.schoolcatchment.com.au%2F%3Fp%3D12301" class="twitter-share-button">Tweet</a></div><a href="http://www.schoolcatchment.com.au/?p=12301&amp;wp_email=1" title="Email This Post" rel="nofollow"><img class="WP-EmailIcon" src="http://www.schoolcatchment.com.au/wp-content/plugins/wp-email/images/email_famfamfam.png" alt="Email This Post" title="Email This Post" style="border: 0px;" /></a>&nbsp;<a href="http://www.schoolcatchment.com.au/?p=12301&amp;wp_email=1" title="Email This Post" rel="nofollow">Email This Post</a></p>
	</div><!-- .entry-content -->

	<footer class="entry-footer">
		<span class="byline"><span class="author vcard"><img alt='' src='http://1.gravatar.com/avatar/a17030ffa68682e1108a598203a0ff8a?s=49&#038;d=mm&#038;r=g' srcset='http://1.gravatar.com/avatar/a17030ffa68682e1108a598203a0ff8a?s=98&#038;d=mm&#038;r=g 2x' class='avatar avatar-49 photo' height='49' width='49' /><span class="screen-reader-text">Author </span> <a class="url fn n" href="http://www.schoolcatchment.com.au/?author=1">admin</a></span></span><span class="posted-on"><span class="screen-reader-text">Posted on </span><a href="http://www.schoolcatchment.com.au/?p=12301" rel="bookmark"><time class="entry-date published" datetime="2019-09-27T23:42:37+11:00">2019-09-27</time><time class="updated" datetime="2019-09-27T23:42:39+11:00">2019-09-27</time></a></span><span class="cat-links"><span class="screen-reader-text">Categories </span><a href="http://www.schoolcatchment.com.au/?cat=7" rel="category">General Education</a>, <a href="http://www.schoolcatchment.com.au/?cat=21" rel="category">Victoria Melbourne Education</a></span>			</footer><!-- .entry-footer -->
</article><!-- #post-12301 -->

<div id="comments" class="comments-area">

	
	
		<div id="respond" class="comment-respond">
		<h2 id="reply-title" class="comment-reply-title">Leave a Reply <small><a rel="nofollow" id="cancel-comment-reply-link" href="/?p=12301#respond" style="display:none;">Cancel reply</a></small></h2>			<form action="http://www.schoolcatchment.com.au/wp-comments-post.php" method="post" id="commentform" class="comment-form" novalidate>
				<p class="comment-notes"><span id="email-notes">Your email address will not be published.</span> Required fields are marked <span class="required">*</span></p><p class="comment-form-comment"><label for="comment">Comment</label> <textarea id="comment" name="comment" cols="45" rows="8" maxlength="65525" required="required"></textarea></p><p class="comment-form-author"><label for="author">Name <span class="required">*</span></label> <input id="author" name="author" type="text" value="" size="30" maxlength="245" required='required' /></p>
<p class="comment-form-email"><label for="email">Email <span class="required">*</span></label> <input id="email" name="email" type="email" value="" size="30" maxlength="100" aria-describedby="email-notes" required='required' /></p>
<p class="comment-form-url"><label for="url">Website</label> <input id="url" name="url" type="url" value="" size="30" maxlength="200" /></p>
<p class="form-submit"><input name="submit" type="submit" id="submit" class="submit" value="Post Comment" /> <input type='hidden' name='comment_post_ID' value='12301' id='comment_post_ID' />
<input type='hidden' name='comment_parent' id='comment_parent' value='0' />
</p><p style="display: none;"><input type="hidden" id="akismet_comment_nonce" name="akismet_comment_nonce" value="bd18ee50ec" /></p><p style="display: none;"><input type="hidden" id="ak_js" name="ak_js" value="166"/></p>			</form>
			</div><!-- #respond -->
	<p class="akismet_comment_form_privacy_notice">This site uses Akismet to reduce spam. <a href="https://akismet.com/privacy/" target="_blank" rel="nofollow noopener">Learn how your comment data is processed</a>.</p>
</div><!-- .comments-area -->

	<nav class="navigation post-navigation" role="navigation">
		<h2 class="screen-reader-text">Post navigation</h2>
		<div class="nav-links"><div class="nav-previous"><a href="http://www.schoolcatchment.com.au/?p=12284" rel="prev"><span class="meta-nav" aria-hidden="true">Previous</span> <span class="screen-reader-text">Previous post:</span> <span class="post-title">Killara Public School In-Person</span></a></div><div class="nav-next"><a href="http://www.schoolcatchment.com.au/?p=12307" rel="next"><span class="meta-nav" aria-hidden="true">Next</span> <span class="screen-reader-text">Next post:</span> <span class="post-title">Interesting Tidbit With Killara Public School</span></a></div></div>
	</nav>
	</main><!-- .site-main -->

	<aside id="content-bottom-widgets" class="content-bottom-widgets" role="complementary">
			<div class="widget-area">
			<section id="text-6" class="widget widget_text">			<div class="textwidget"><p><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script><br />
<!-- SchoolCatchment Sidebar --><br />
<ins class="adsbygoogle"
     style="display:block"
     data-ad-client="ca-pub-3991833096608295"
     data-ad-slot="1042533966"
     data-ad-format="auto"></ins><br />
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></p>
</div>
		</section>		<section id="recent-posts-4" class="widget widget_recent_entries">		<h2 class="widget-title">Recent Posts</h2>		<ul>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12338">Australia Top ICSEA Ranked School For 2018</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12314">Balgowlah North Public School In-Person</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12311">Wentworth Point Public School</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12307">Interesting Tidbit With Killara Public School</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12301">Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government)</a>
									</li>
					</ul>
		</section><section id="categories-4" class="widget widget_categories"><h2 class="widget-title">Categories</h2>		<ul>
				<li class="cat-item cat-item-13"><a href="http://www.schoolcatchment.com.au/?cat=13">Australian Capital Territory (ACT) Public School Priority Enrolment Area (Catchment)</a>
</li>
	<li class="cat-item cat-item-24"><a href="http://www.schoolcatchment.com.au/?cat=24">Australian School Funding</a>
</li>
	<li class="cat-item cat-item-7"><a href="http://www.schoolcatchment.com.au/?cat=7">General Education</a>
</li>
	<li class="cat-item cat-item-5"><a href="http://www.schoolcatchment.com.au/?cat=5">General Mapping</a>
</li>
	<li class="cat-item cat-item-8"><a href="http://www.schoolcatchment.com.au/?cat=8">Misc</a>
</li>
	<li class="cat-item cat-item-9"><a href="http://www.schoolcatchment.com.au/?cat=9">NSW And Sydney High School Catchment Map</a>
</li>
	<li class="cat-item cat-item-2"><a href="http://www.schoolcatchment.com.au/?cat=2">NSW And Sydney Public School Catchment Map</a>
</li>
	<li class="cat-item cat-item-16"><a href="http://www.schoolcatchment.com.au/?cat=16">Perth Primary School Local Intake Area (Catchment) Map</a>
</li>
	<li class="cat-item cat-item-19"><a href="http://www.schoolcatchment.com.au/?cat=19">Queensland School Catchment</a>
</li>
	<li class="cat-item cat-item-4"><a href="http://www.schoolcatchment.com.au/?cat=4">School by School Introduction</a>
</li>
	<li class="cat-item cat-item-14"><a href="http://www.schoolcatchment.com.au/?cat=14">School Children Parenting experience</a>
</li>
	<li class="cat-item cat-item-18"><a href="http://www.schoolcatchment.com.au/?cat=18">South Australia Public School Zone</a>
</li>
	<li class="cat-item cat-item-17"><a href="http://www.schoolcatchment.com.au/?cat=17">South Australia School Zone Catchment Map</a>
</li>
	<li class="cat-item cat-item-3"><a href="http://www.schoolcatchment.com.au/?cat=3">Sydney Public School Information</a>
</li>
	<li class="cat-item cat-item-23"><a href="http://www.schoolcatchment.com.au/?cat=23">Tasmania School Zone and Information</a>
</li>
	<li class="cat-item cat-item-6"><a href="http://www.schoolcatchment.com.au/?cat=6">Top Sydney/NSW Public School</a>
</li>
	<li class="cat-item cat-item-1"><a href="http://www.schoolcatchment.com.au/?cat=1">Uncategorized</a>
</li>
	<li class="cat-item cat-item-21"><a href="http://www.schoolcatchment.com.au/?cat=21">Victoria Melbourne Education</a>
</li>
	<li class="cat-item cat-item-11"><a href="http://www.schoolcatchment.com.au/?cat=11">Victoria, Melbourne Primary School Zone Map</a>
</li>
	<li class="cat-item cat-item-12"><a href="http://www.schoolcatchment.com.au/?cat=12">Victoria, Melbourne Secondary School Zone Map</a>
</li>
	<li class="cat-item cat-item-15"><a href="http://www.schoolcatchment.com.au/?cat=15">Western Australia Primary School Local Intake Area (Catchment) Map</a>
</li>
	<li class="cat-item cat-item-22"><a href="http://www.schoolcatchment.com.au/?cat=22">Western Australia School Intake Area</a>
</li>
		</ul>
			</section><section id="recent-comments-4" class="widget widget_recent_comments"><h2 class="widget-title">Recent Comments</h2><ul id="recentcomments"><li class="recentcomments"><span class="comment-author-link">admin</span> on <a href="http://www.schoolcatchment.com.au/?p=6050#comment-31963">Top 100 Ranking Victoria Melbourne Primary Schools For The Year Of 2016 (Government/Public)</a></li><li class="recentcomments"><span class="comment-author-link">Prakash</span> on <a href="http://www.schoolcatchment.com.au/?p=6050#comment-31948">Top 100 Ranking Victoria Melbourne Primary Schools For The Year Of 2016 (Government/Public)</a></li><li class="recentcomments"><span class="comment-author-link">admin</span> on <a href="http://www.schoolcatchment.com.au/?p=1401#comment-31841">Canberra, ACT College Priority Enrolment Area Year 11 to 12 (Catchment) Map</a></li><li class="recentcomments"><span class="comment-author-link">admin</span> on <a href="http://www.schoolcatchment.com.au/?p=5125#comment-31840">Cammeraygal High School</a></li><li class="recentcomments"><span class="comment-author-link">Tina Vandervelde</span> on <a href="http://www.schoolcatchment.com.au/?p=1401#comment-31784">Canberra, ACT College Priority Enrolment Area Year 11 to 12 (Catchment) Map</a></li></ul></section><section id="text-7" class="widget widget_text">			<div class="textwidget"><p><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script><br />
<!-- SchoolCatchment Sidebar --><br />
<ins class="adsbygoogle"
     style="display:block"
     data-ad-client="ca-pub-3991833096608295"
     data-ad-slot="1042533966"
     data-ad-format="auto"></ins><br />
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></p>
</div>
		</section>		</div><!-- .widget-area -->
	
			<div class="widget-area">
			<section id="text-13" class="widget widget_text">			<div class="textwidget"><p><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script><br />
<!-- SchoolCatchment Sidebar --><br />
<ins class="adsbygoogle" style="display: block;" data-ad-client="ca-pub-3991833096608295" data-ad-slot="1042533966" data-ad-format="auto" data-full-width-responsive="true"></ins><br />
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script><code></p>
</div>
		</section>		</div><!-- .widget-area -->
	</aside><!-- .content-bottom-widgets -->

</div><!-- .content-area -->


	<aside id="secondary" class="sidebar widget-area" role="complementary">
		<section id="text-18" class="widget widget_text">			<div class="textwidget"><p><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script><br />
<!-- SchoolCatchment Sidebar --><br />
<ins class="adsbygoogle" style="display: block;" data-ad-client="ca-pub-3991833096608295" data-ad-slot="1042533966" data-ad-format="auto" data-full-width-responsive="true"></ins><br />
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></p>
</div>
		</section><section id="text-19" class="widget widget_text"><h2 class="widget-title">Google Adsense</h2>			<div class="textwidget"><p><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script><br />
<!-- SchoolCatchment Sidebar --><br />
<ins class="adsbygoogle" style="display: block;" data-ad-client="ca-pub-3991833096608295" data-ad-slot="1042533966" data-ad-format="auto" data-full-width-responsive="true"></ins><br />
<script>
(adsbygoogle = window.adsbygoogle || []).push({});
</script></p>
</div>
		</section><section id="search-3" class="widget widget_search">
<form role="search" method="get" class="search-form" action="http://www.schoolcatchment.com.au/">
	<label>
		<span class="screen-reader-text">Search for:</span>
		<input type="search" class="search-field" placeholder="Search &hellip;" value="" name="s" />
	</label>
	<button type="submit" class="search-submit"><span class="screen-reader-text">Search</span></button>
</form>
</section><section id="categories-7" class="widget widget_categories"><h2 class="widget-title">Categories</h2><form action="http://www.schoolcatchment.com.au" method="get"><label class="screen-reader-text" for="cat">Categories</label><select  name='cat' id='cat' class='postform' >
	<option value='-1'>Select Category</option>
	<option class="level-0" value="13">Australian Capital Territory (ACT) Public School Priority Enrolment Area (Catchment)</option>
	<option class="level-0" value="24">Australian School Funding</option>
	<option class="level-0" value="7">General Education</option>
	<option class="level-0" value="5">General Mapping</option>
	<option class="level-0" value="8">Misc</option>
	<option class="level-0" value="9">NSW And Sydney High School Catchment Map</option>
	<option class="level-0" value="2">NSW And Sydney Public School Catchment Map</option>
	<option class="level-0" value="16">Perth Primary School Local Intake Area (Catchment) Map</option>
	<option class="level-0" value="19">Queensland School Catchment</option>
	<option class="level-0" value="4">School by School Introduction</option>
	<option class="level-0" value="14">School Children Parenting experience</option>
	<option class="level-0" value="18">South Australia Public School Zone</option>
	<option class="level-0" value="17">South Australia School Zone Catchment Map</option>
	<option class="level-0" value="3">Sydney Public School Information</option>
	<option class="level-0" value="23">Tasmania School Zone and Information</option>
	<option class="level-0" value="6">Top Sydney/NSW Public School</option>
	<option class="level-0" value="1">Uncategorized</option>
	<option class="level-0" value="21">Victoria Melbourne Education</option>
	<option class="level-0" value="11">Victoria, Melbourne Primary School Zone Map</option>
	<option class="level-0" value="12">Victoria, Melbourne Secondary School Zone Map</option>
	<option class="level-0" value="15">Western Australia Primary School Local Intake Area (Catchment) Map</option>
	<option class="level-0" value="22">Western Australia School Intake Area</option>
</select>
</form>
<script type='text/javascript'>
/* <![CDATA[ */
(function() {
	var dropdown = document.getElementById( "cat" );
	function onCatChange() {
		if ( dropdown.options[ dropdown.selectedIndex ].value > 0 ) {
			dropdown.parentNode.submit();
		}
	}
	dropdown.onchange = onCatChange;
})();
/* ]]> */
</script>

			</section>		<section id="recent-posts-7" class="widget widget_recent_entries">		<h2 class="widget-title">Recent Posts</h2>		<ul>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12338">Australia Top ICSEA Ranked School For 2018</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12314">Balgowlah North Public School In-Person</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12311">Wentworth Point Public School</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12307">Interesting Tidbit With Killara Public School</a>
									</li>
											<li>
					<a href="http://www.schoolcatchment.com.au/?p=12301">Top 500 Ranking Victoria Melbourne Primary School For The Year Of 2018 (Government and Non-government)</a>
									</li>
					</ul>
		</section><section id="pages-3" class="widget widget_pages"><h2 class="widget-title">Pages</h2>		<ul>
			<li class="page_item page-item-2"><a href="http://www.schoolcatchment.com.au/?page_id=2">About us</a></li>
<li class="page_item page-item-1051"><a href="http://www.schoolcatchment.com.au/?page_id=1051">ACT Public School Priority Enrolment Area (Catchment) Map</a></li>
<li class="page_item page-item-36"><a href="http://www.schoolcatchment.com.au/?page_id=36">NSW/Sydney Public School Catchment Map</a></li>
<li class="page_item page-item-5654"><a href="http://www.schoolcatchment.com.au/?page_id=5654">Queensland School Catchment Map</a></li>
<li class="page_item page-item-5367"><a href="http://www.schoolcatchment.com.au/?page_id=5367">South Australia Public School Zone Map</a></li>
<li class="page_item page-item-488"><a href="http://www.schoolcatchment.com.au/?page_id=488">Sydney High School Catchment Map</a></li>
<li class="page_item page-item-5985"><a href="http://www.schoolcatchment.com.au/?page_id=5985">Top Ranking Australian Schools</a></li>
<li class="page_item page-item-825"><a href="http://www.schoolcatchment.com.au/?page_id=825">Victoria/Melbourne Public School Zone Map</a></li>
<li class="page_item page-item-1148"><a href="http://www.schoolcatchment.com.au/?page_id=1148">Western Australia School Local Intake Area (Catchment) Map</a></li>
		</ul>
			</section><section id="custom_html-4" class="widget_text widget widget_custom_html"><h2 class="widget-title">Google Adsense</h2><div class="textwidget custom-html-widget"><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<!-- SchoolCatchment Sidebar -->

<ins class="adsbygoogle"

     style="display:block"

     data-ad-client="ca-pub-3991833096608295"

     data-ad-slot="1042533966"

     data-ad-format="auto"

     data-full-width-responsive="true"></ins>

<script>

(adsbygoogle = window.adsbygoogle || []).push({});

</script></div></section><section id="custom_html-5" class="widget_text widget widget_custom_html"><h2 class="widget-title">Google Adsense</h2><div class="textwidget custom-html-widget"><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<!-- SchoolCatchment Sidebar -->

<ins class="adsbygoogle"

     style="display:block"

     data-ad-client="ca-pub-3991833096608295"

     data-ad-slot="1042533966"

     data-ad-format="auto"

     data-full-width-responsive="true"></ins>

<script>

(adsbygoogle = window.adsbygoogle || []).push({});

</script></div></section><section id="custom_html-6" class="widget_text widget widget_custom_html"><h2 class="widget-title">Google Adsense</h2><div class="textwidget custom-html-widget"><script async src="//pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<!-- SchoolCatchment Sidebar -->

<ins class="adsbygoogle"

     style="display:block"

     data-ad-client="ca-pub-3991833096608295"

     data-ad-slot="1042533966"

     data-ad-format="auto"

     data-full-width-responsive="true"></ins>

<script>

(adsbygoogle = window.adsbygoogle || []).push({});

</script></div></section><section id="custom_html-8" class="widget_text widget widget_custom_html"><h2 class="widget-title">Google Adsense</h2><div class="textwidget custom-html-widget"><script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<!-- SchoolCatchment Sidebar -->

<ins class="adsbygoogle"

     style="display:block"

     data-ad-client="ca-pub-3991833096608295"

     data-ad-slot="1042533966"

     data-ad-format="auto"

     data-full-width-responsive="true"></ins>

<script>

     (adsbygoogle = window.adsbygoogle || []).push({});

</script></div></section>	</aside><!-- .sidebar .widget-area -->

		</div><!-- .site-content -->

		<footer id="colophon" class="site-footer" role="contentinfo">
							<nav class="main-navigation" role="navigation" aria-label="Footer Primary Menu">
					<div class="menu-main-menu-container"><ul id="menu-main-menu-1" class="primary-menu"><li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11854"><a href="http://www.schoolcatchment.com.au/?page_id=5985">Top Ranking Australian Schools</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11855"><a href="http://www.schoolcatchment.com.au/?page_id=5654">Queensland School Catchment Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11856"><a href="http://www.schoolcatchment.com.au/?page_id=5367">South Australia Public School Zone Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11857"><a href="http://www.schoolcatchment.com.au/?page_id=1148">Western Australia School Local Intake Area (Catchment) Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11858"><a href="http://www.schoolcatchment.com.au/?page_id=1051">ACT Public School Priority Enrolment Area (Catchment) Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11859"><a href="http://www.schoolcatchment.com.au/?page_id=825">Victoria/Melbourne Public School Zone Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11860"><a href="http://www.schoolcatchment.com.au/?page_id=488">Sydney High School Catchment Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11861"><a href="http://www.schoolcatchment.com.au/?page_id=36">NSW/Sydney Public School Catchment Map</a></li>
<li class="menu-item menu-item-type-post_type menu-item-object-page menu-item-11862"><a href="http://www.schoolcatchment.com.au/?page_id=2">About us</a></li>
</ul></div>				</nav><!-- .main-navigation -->
			
			
			<div class="site-info">
								<span class="site-title"><a href="http://www.schoolcatchment.com.au/" rel="home">Australian Public School Information</a></span>
								<a href="https://wordpress.org/" class="imprint">
					Proudly powered by WordPress				</a>
			</div><!-- .site-info -->
		</footer><!-- .site-footer -->
	</div><!-- .site-inner -->
</div><!-- .site -->

<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>

<!-- SchoolCatchment Sidebar -->

<ins class="adsbygoogle"

     style="display:block"

     data-ad-client="ca-pub-3991833096608295"

     data-ad-slot="1042533966"

     data-ad-format="auto"

     data-full-width-responsive="true"></ins>

<script>

     (adsbygoogle = window.adsbygoogle || []).push({});

</script><script type='text/javascript'>
/* <![CDATA[ */
var emailL10n = {"ajax_url":"http:\/\/www.schoolcatchment.com.au\/wp-admin\/admin-ajax.php","max_allowed":"5","text_error":"The Following Error Occurs:","text_name_invalid":"- Your Name is empty\/invalid","text_email_invalid":"- Your Email is empty\/invalid","text_remarks_invalid":"- Your Remarks is invalid","text_friend_names_empty":"- Friend Name(s) is empty","text_friend_name_invalid":"- Friend Name is empty\/invalid: ","text_max_friend_names_allowed":"- Maximum 5 Friend Names allowed","text_friend_emails_empty":"- Friend Email(s) is empty","text_friend_email_invalid":"- Friend Email is invalid: ","text_max_friend_emails_allowed":"- Maximum 5 Friend Emails allowed","text_friends_tally":"- Friend Name(s) count does not tally with Friend Email(s) count","text_image_verify_empty":"- Image Verification is empty"};
/* ]]> */
</script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/plugins/wp-email/email-js.js?ver=2.67.6'></script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/js/skip-link-focus-fix.js?ver=20160816'></script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-includes/js/comment-reply.min.js?ver=5.2.4'></script>
<script type='text/javascript'>
/* <![CDATA[ */
var screenReaderText = {"expand":"expand child menu","collapse":"collapse child menu"};
/* ]]> */
</script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/themes/twentysixteen/js/functions.js?ver=20181230'></script>
<script type='text/javascript'>
/* <![CDATA[ */
var FB_WP=FB_WP||{};FB_WP.queue={_methods:[],flushed:false,add:function(fn){FB_WP.queue.flushed?fn():FB_WP.queue._methods.push(fn)},flush:function(){for(var fn;fn=FB_WP.queue._methods.shift();){fn()}FB_WP.queue.flushed=true}};window.fbAsyncInit=function(){FB.init({"xfbml":true});if(FB_WP && FB_WP.queue && FB_WP.queue.flush){FB_WP.queue.flush()}}
/* ]]> */
</script>
<script type="text/javascript">(function(d,s,id){var js,fjs=d.getElementsByTagName(s)[0];if(d.getElementById(id)){return}js=d.createElement(s);js.id=id;js.src="http:\/\/connect.facebook.net\/en_US\/all.js";fjs.parentNode.insertBefore(js,fjs)}(document,"script","facebook-jssdk"));</script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-includes/js/wp-embed.min.js?ver=5.2.4'></script>
<script type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/plugins/tablepress/js/jquery.datatables.min.js?ver=1.9.2'></script>
<script type='text/javascript'>
/* <![CDATA[ */
window.twttr=(function(w){t=w.twttr||{};t._e=[];t.ready=function(f){t._e.push(f);};return t;}(window));
/* ]]> */
</script>
<script type="text/javascript" id="twitter-wjs" async defer src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>
<script async="async" type='text/javascript' src='http://www.schoolcatchment.com.au/wp-content/plugins/akismet/_inc/form.js?ver=4.1.2'></script>
<script type="text/javascript">
jQuery(document).ready(function($){
$('#tablepress-233').dataTable({"order":[],"orderClasses":false,"stripeClasses":["even","odd"],"pagingType":"simple"});
});
</script><div id="fb-root"></div></body>
</html>

'''


In [83]:
#inputFile1 = open('secondary.txt', 'r', encoding='utf-8').read()
# separate each section with "<hr />"
pattern1=inputFile1.split("<hr />")
del pattern1[584]
# create dict
sec_dict={}
for each_p in pattern1:
    sec_info = re.findall('(?:<div>)(.*?)(?:</div>)', each_p)
    if sec_info[0] not in sec_dict.keys():
        sec_dict[sec_info[0]]=sec_info[1]
    elif sec_info[0] in sec_dict.keys():
        old_rank=sec_dict[sec_info[0]]
        # if old rank number is smaller, keep the old rank
        if old_rank<sec_info[1]:
            sec_dict[sec_info[0]]=old_rank
        elif old_rank>sec_info[1]:
            sec_dict[sec_info[0]]=sec_info[1]            
print(sec_dict)

{'Academy of Mary Immaculate': '144', 'Adass Israel School': '534', 'Advance College of Education': '535', 'Aitken College': '194', 'Al Siraat College': '280', 'Al-Taqwa College': '400', 'Albert Park College': '131', 'Albury Wodonga Comm College': '536', 'Alexandra Secondary College': '481', 'Alia College': '500', 'Alice Miller School': '127', 'Alkira Secondary College': '328', 'Alphington Grammar School': '86', 'Antonine College': '482', 'Apollo Bay P-12 College': '18', 'Aquinas College': '151', 'Ararat Secondary College': '486', 'Ashwood High School': '241', 'Assumption College': '301', 'Auburn High School': '167', 'Australian Internatl Academy': '501', 'Ave Maria College': '111', 'Avila College': '102', 'Bacchus Marsh College': '496', 'Bacchus Marsh Grammar': '147', 'Baimbridge College': '452', 'Bairnsdale Secondary College': '513', 'Balcombe Grammar School': '156', 'Ballarat Christian College': '408', 'Ballarat Clarendon College': '12', 'Ballarat Grammar': '84', 'Ballarat High Scho

In [84]:
# use close school list to match the primary school name in ranking dict
final_sec_rank=[]
for i in range(len(closest_secondary_school)):
    # set n=1, return one match, butoff=0.85
    match=difflib.get_close_matches(closest_secondary_school[i], sec_dict.keys(), n=1, cutoff=0.85)
    print(closest_secondary_school[i],match)
    if match !=[]:
        final_sec_rank.append(sec_dict[match[0]])
    elif match==[]:
        final_sec_rank.append('not ranked')
    
final_sec_rank

Kew High School ['Kew High School']
Victoria University Secondary College []
Mount Alexander 7-12 College ['Mount Alexander 7-12 Coll']
Richmond High School []
Ashwood High School ['Ashwood High School']
Victorian College Of The Arts Secondary School []
Footscray City College ['Footscray City College']
Balwyn High School ['Balwyn High School']
Gilmore College For Girls ['Gilmore College For Girls']
Frankston High School ['Frankston High School']
Korowa Anglican Girls' School ["Korowa Anglican Girls' School"]
Our Lady of the Sacred Heart College ['Our Lady of Sacred Heart Coll']
Bundoora Secondary College ['Bundoora Secondary College']
Brunswick Secondary College ['Brunswick Secondary College']
Bayside P-12 College ['Bayside P-12 College']
St John's College Preston []
University High School ['University High School']
Leibler Yavneh College ['Leibler Yavneh College']
Highvale Secondary College ['Highvale Secondary College']
McKinnon Secondary College ['McKinnon Secondary College']
East P

Albert Park College ['Albert Park College']
Staughton College ['Staughton College']
Williamstown High School ['Williamstown High School']
St Columba's College ["St Columba's College"]
Carranballac P-9 College []
Ivanhoe Girls' Grammar School ["Ivanhoe Girls' Grammar School"]
Caroline Chisholm Catholic College ['Caroline Chisholm Catholic Coll']
Penleigh & Essendon Grammar School []
Strathmore Secondary College ['Strathmore Secondary College']
Mordialloc College ['Mordialloc College']
Gilmore College For Girls ['Gilmore College For Girls']
Scotch College ['Scotch College']
Fintona Girls' School ['Fintona Girls School']
Springside West Secondary College []
Albert Park College ['Albert Park College']
Maribyrnong Secondary College ['Maribyrnong Sec College']
Catholic Regional College ['Catholic Regional College']
St Mary's Coptic Orthodox College ["St Mary's Coptic Orth College"]
Emmanuel College ['Emmanuel College']
Marcellin College ['Marcellin College']
Buckley Park College ['Buckley Pa

Bentleigh Secondary College ['Bentleigh Secondary College']
Pascoe Vale Girls Secondary College ['Pascoe Vale Girls Sec College']
Reservoir High School ['Reservoir High School']
Maribyrnong Secondary College ['Maribyrnong Sec College']
Fitzroy High School ['Fitzroy High School']
Brunswick Secondary College ['Brunswick Secondary College']
Lowther Hall Anglican Grammar School []
Altona College []
McKinnon Secondary College ['McKinnon Secondary College']
Emmanuel College ['Emmanuel College']
Albert Park College ['Albert Park College']
William Ruthven Secondary College ['William Ruthven Sec College']
Melbourne Girls College ["Melbourne Girls' College"]
St John's College Preston []
East Preston Islamic College ['East Preston Islamic College']
Albert Park College ['Albert Park College']
Braybrook College ['Braybrook College']
Kingswood College ['Kingswood College']
Albert Park College ['Albert Park College']
Forest Hill College ['Forest Hill College']
Korowa Anglican Girls' School ["Korowa A

Coburg High School ['Coburg High School']
Lynall Hall Community School ['Lynall Hall Community School']
Antonine College ['Antonine College']
Bentleigh Secondary College ['Bentleigh Secondary College']
Thornbury High School ['Thornbury High School']
Albert Park College ['Albert Park College']
Glenroy Private []
The King David School ['The King David School']
Gilmore College For Girls ['Gilmore College For Girls']
Bayside P-12 College ['Bayside P-12 College']
Our Lady of Mercy College ['Our Lady of Mercy College']
De La Salle College ['De La Salle College']
Auburn High School ['Auburn High School']
Santa Maria College ['Santa Maria College']
Lowther Hall Anglican Grammar School []
St Leonard's College ["St Leonard's College"]
Patterson River Secondary College ['Patterson River Sec College']
Gilmore College For Girls ['Gilmore College For Girls']
Melbourne Girls Grammar ['Melbourne Girls Grammar']
Elwood College ['Elwood College']
Preston High School ['Frankston High School']
Box Hill Se

['124',
 'not ranked',
 '348',
 'not ranked',
 '241',
 'not ranked',
 '166',
 '44',
 '387',
 '87',
 '13',
 '73',
 '531',
 '211',
 '243',
 'not ranked',
 '67',
 '28',
 '138',
 '48',
 '491',
 '558',
 '288',
 'not ranked',
 '123',
 '270',
 '32',
 '558',
 '178',
 '119',
 '243',
 '371',
 '166',
 '409',
 '48',
 '318',
 '50',
 '81',
 '447',
 '139',
 '261',
 '346',
 '22',
 '243',
 '56',
 '211',
 '123',
 '435',
 'not ranked',
 '230',
 '225',
 '435',
 '133',
 '169',
 '67',
 '230',
 '96',
 '37',
 '22',
 '175',
 '270',
 '459',
 '235',
 '387',
 '559',
 '126',
 '50',
 '248',
 '37',
 '46',
 '307',
 '130',
 '416',
 'not ranked',
 'not ranked',
 '125',
 '371',
 '387',
 '2',
 '1',
 '166',
 'not ranked',
 '435',
 '123',
 '307',
 '13',
 '126',
 '6',
 '176',
 '304',
 '213',
 '243',
 '248',
 'not ranked',
 '491',
 'not ranked',
 '459',
 '9',
 'not ranked',
 '530',
 '511',
 '139',
 '94',
 '2',
 '371',
 '430',
 '2',
 '2',
 '61',
 '244',
 '270',
 '435',
 'not ranked',
 'not ranked',
 'not ranked',
 '131',
 'no

In [85]:
# There are 93 secondary school cannot find a match
final_sec_rank.count('not ranked')

93

In [86]:
gr['secondary_school_ranking']= final_sec_rank
gr.head()

,ID,Address,Rooms,Type,Price,Date,Bathroom,Car,LandSize,Age,...,crime_A_average,crime_B_average,crime_C_average,closest_primary_school,distance_to_closest_primary,primary_school_ranking,closest_secondary_school,distance_to_closest_secondary,secondary_school_ranking,Points
0,ID973006338,2/14 Hatfield St,2,u,725000.0,27/05/2017,1.0,1.0,134.0,49.0,...,959,4476,324,St Bede's School,0.503420,442,Kew High School,0.206565,124,POINT (145.0657 -37.79541)
1,ID281275681,5 Hedgerow Ct,3,h,565000.0,23/09/2017,1.0,2.0,619.0,69.0,...,119,452,22,Albanvale Primary School,0.526422,not ranked,Victoria University Secondary College,1.600500,not ranked,POINT (144.76863 -37.74521)
2,ID423911400,96 Bryant St,2,h,865500.0,3/09/2016,1.0,1.0,168.0,114.0,...,1923,8979,620,St Brendan's School,0.232892,not ranked,Mount Alexander 7-12 College,0.160850,348,POINT (144.9339 -37.7835)
3,ID249026802,320 Church St,3,h,1651000.0,24/09/2016,2.0,1.0,192.0,119.0,...,824,1890,159,Richmond West Primary School,0.877476,452,Richmond High School,0.566940,not ranked,POINT (144.9985 -37.8212)
4,ID687137769,4/58 Bath Rd,2,u,671000.0,4/03/2017,1.0,1.0,0.0,59.0,...,2047,9171,742,Ashburton Primary School,0.878217,106,Ashwood High School,1.617907,241,POINT (145.0889 -37.8555)


In [87]:
# generate output file
gr_out = gr.to_csv('Group108_solution.csv',index=False)